# MD Package

## 1.  Load Transactional Data, SalesForce Data, Mavenlink Data, load packages, and convert to Dataframes

I am using three reports from Netsuite to populate this data:  Lists/Employees --> Run report without filters all set to "All".

Search in Netsuite search bar usings "sea" for Adaptive Transaction Search - P&L - Jeremy

Search in Netsuite search bar using "sea" for Mavenlink Time Audit

In Salesforce, use Ginny's link, but filter for "ALL" in Opportunity Record Type.  Also include Revenue Date:  greater or equal to LAST FISCAL YEAR.

![title](img/picture.jpg)

In [1]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl import load_workbook
from openpyxl.styles import Font
from openpyxl.styles import Alignment
from openpyxl.styles.borders import Border, Side


In [2]:
dfHistoric = pd.read_excel('../Data/raw/FY19&FY20_Detail_Trans.xlsx')

In [3]:
dfFY18 = pd.read_excel('../Data/raw/ADAPTIVETransactionSearchPLResultsFY18.xlsx')

In [4]:
dfGrantBalances = pd.read_excel('../Data/raw/Grant_Balance_Report_MARCH_V4_LOAD.xlsx')

In [5]:
#Get this from Mavenlink, Custom Insights: Sharepoint Reports, Project List for MDs:  Export to XLSX, uncheck "Keep attribute cells merged" and "Include active filters in each report"

dfMAVELINKACTIVEPROJECTS = pd.read_excel('../Data/raw/Sharepoint Reports 4-14-2021 0701am.xlsx')

In [6]:
dfHistoric = pd.concat([dfFY18, dfHistoric], ignore_index=True)

In [7]:
from xml.sax import ContentHandler, parse

# Reference https://goo.gl/KaOBG3
class ExcelHandler(ContentHandler):
    def __init__(self):
        self.chars = [  ]
        self.cells = [  ]
        self.rows = [  ]
        self.tables = [  ]
    def characters(self, content):
        self.chars.append(content)
    def startElement(self, name, atts):
        if name=="Cell":
            self.chars = [  ]
        elif name=="Row":
            self.cells=[  ]
        elif name=="Table":
            self.rows = [  ]
    def endElement(self, name):
        if name=="Cell":
            self.cells.append(''.join(self.chars))
        elif name=="Row":
            self.rows.append(self.cells)
        elif name=="Table":
            self.tables.append(self.rows)

excelHandler = ExcelHandler()
parse('../Data/raw/ADAPTIVETransactionSearchPLJeremyResults145.xls', excelHandler)
dfFY21 = pd.DataFrame(excelHandler.tables[0][1:], columns=excelHandler.tables[0][0])

len(dfFY21)

108482

In [8]:
#Search "Mavenlink Time Audit", filter it to start at the beginning of the fiscal year.

#excelHandler = ExcelHandler()
#parse('../Data/raw/MavenlinkTimeAuditResults905.xls', excelHandler)
#dfMavenlink = pd.DataFrame(excelHandler.tables[0][1:], columns=excelHandler.tables[0][0])

In [9]:
excelHandler = ExcelHandler()
parse('../Data/raw/Employees588.xls', excelHandler)
employee_list = pd.DataFrame(excelHandler.tables[0][1:], columns=excelHandler.tables[0][0])


In [10]:
dfSalesforce = pd.read_html('../Data/raw/report1618858455808.xls')
dfSalesforce = dfSalesforce[0]

In [11]:
dfSalesforceFFS = pd.read_html('../Data/raw/report1618858444686.xls')
dfSalesforceFFS = dfSalesforceFFS[0]

In [12]:
dfSalesforceGrantContracts = pd.read_html('../Data/raw/report1620395246783.xls')
dfSalesforceGrantContracts = dfSalesforceGrantContracts[0]

In [13]:
#To get the FFS Balances and Whitney's data from Netsuite: RMI Fee for Service Contract Balances

excelHandler = ExcelHandler()
parse('../Data/raw/RMIFeeforServiceContractBalancesResults927.xls', excelHandler)
dfFFSData = pd.DataFrame(excelHandler.tables[0][1:], columns=excelHandler.tables[0][0])

In [14]:
#"FFS Projects" from Netsuite
excelHandler = ExcelHandler()
parse('../Data/raw/FFSProjectsResults94.xls', excelHandler)
dfFFSData2 = pd.DataFrame(excelHandler.tables[0][1:], columns=excelHandler.tables[0][0])

In [15]:
#To get the deferred Revenue data for FFS from Netsuite:  RMI Transaction Search - Revenue Summary

excelHandler = ExcelHandler()
parse('../Data/raw/RMITransactionSearchRevenueSummaryResults277.xls', excelHandler)
dfFFSIncludingDeferred = pd.DataFrame(excelHandler.tables[0][1:], columns=excelHandler.tables[0][0])

In [16]:
df1 = pd.concat([dfHistoric, dfFY21], ignore_index=True)


## 2. Data Manipulation

In this section I start manipulting these datasets into somethign more digestable for the end user.  The main manipulation I am doing in this seciton is pulling information out of the "Memo" field that users may want to see seperately.

In [17]:
#EmployeeNameandID = employee_list[['Name', 'External ID', 'Job Title']]

In [18]:
#EmployeeNameandID['External ID'].replace('', np.nan, inplace=True)

In [19]:
#len(EmployeeNameandID)

In [20]:
#EmployeeNameandID.to_excel('originallist.xlsx')

In [21]:
#employee_id_list = employee_list['External ID'].tolist()

In [22]:
#employee_id_list = list(filter(None, employee_id_list))

In [23]:
#print(len(employee_id_list))

In [24]:
df1['Memo'] = df1['Memo'].astype('string')

In [25]:
df1.dtypes

Date                               object
Period                             object
GL Account                         object
Document Number                    object
Program                            object
Initiative                         object
Project                            object
Restriction                        object
Grant                              object
Amount                             object
Vendor Name                        object
Employee Name                      object
Memo                               string
Amount (Credit)                    object
Amount (Debit)                     object
Client                             object
Transaction Number                 object
Type                               object
Vendor Rate                        object
Vendor Contract Expiration Date    object
dtype: object

In [26]:
#all_ids = '|'.join(employee_id_list)

In [27]:
def return_first(x):
    try:
        y = x[0]
    except:
        y = ""
    return y

In [28]:
#df1['External ID'] = df1['Memo'].str.findall(all_ids).apply(return_first)

In [29]:
#employee_name_list = employee_list['Name'].tolist()

In [30]:
#employee_name_list = list(filter(None, employee_name_list))

In [31]:
#all_names = '|'.join(employee_name_list)

In [32]:
#all_names = all_names + "|Agnis Li|Anna Ledden|Greg Hales|Rachel Hohensee|Li Ma"

In [33]:
#df1['Match ID Names'] = df1['Memo'].str.findall(all_names).apply(return_first)

In [34]:
#df1 = pd.merge(df1, EmployeeNameandID, on = 'External ID', how='left')

In [35]:
#df1['Match ID Names'].replace('', np.nan, inplace=True)

In [36]:
#df1['Name'].replace('', np.nan, inplace=True)

In [37]:
#df1['Name'].fillna(df1['Match ID Names'], inplace=True)

In [38]:
#df1['Name'].replace('Agnis Li', 'Agnes Li', inplace=True)
#df1['Name'].replace('Anna Ledden', 'Anna  Ledden', inplace=True)
#df1['Name'].replace('Rachel Hohensee', 'Rachel T Hohensee', inplace=True)

## Salesforce Data Manipulation

In [39]:
dfSalesforce['Revenue Status'].replace('', np.nan, inplace=True)
dfSalesforce['Initiative'].replace('', np.nan, inplace=True)
dfSalesforce['Revenue Amount'].replace('', np.nan, inplace=True)
dfSalesforce['RMI Grant Name'].replace('', np.nan, inplace=True)
dfSalesforce['Grant Term - Start Date'].replace('', np.nan, inplace=True)
dfSalesforce['Grant Term - End Date'].replace('', np.nan, inplace=True)

In [40]:
dfSalesforce['Revenue Status'].fillna("na", inplace=True)
dfSalesforce['Initiative'].fillna("na", inplace=True)
dfSalesforce['Revenue Amount'].fillna(0, inplace=True)
dfSalesforce['RMI Grant Name'].fillna("na", inplace=True)
dfSalesforce['Grant Term - Start Date'].fillna("na", inplace=True)
dfSalesforce['Grant Term - End Date'].fillna("na", inplace=True)

In [41]:
dfSalesforceFFS['Amount'].replace('', np.nan, inplace=True)
dfSalesforceFFS['Relationship Manager'].replace('', np.nan, inplace=True)
dfSalesforceFFS['Initiative'].replace('', np.nan, inplace=True)

In [42]:
dfSalesforceFFS['Amount'].fillna(0, inplace=True)
dfSalesforceFFS['Relationship Manager'].fillna('na', inplace=True)
dfSalesforceFFS['Initiative'].fillna('na', inplace=True)

In [43]:
dfSalesforceGrantContracts["Documents Link"] = dfSalesforceGrantContracts['Essential Documents Link'].str.findall(r'\<a href=(.*?)\ target').apply(return_first)


In [44]:
dfSalesforceGrantContractsNS = dfSalesforceGrantContracts[['RMI Grant Name', 'Documents Link']]

In [45]:
dfSalesforceGrantContractsNS = dfSalesforceGrantContractsNS.dropna()
dfSalesforceGrantContractsNS = dfSalesforceGrantContractsNS.rename(columns={'RMI Grant Name':'Grant'})



In [46]:

dfSalesforceGrantContractsNS = dfSalesforceGrantContractsNS.drop_duplicates(subset='Grant', keep="first")

In [47]:
dfSalesforceGrantContractsNS['Documents Link'] = dfSalesforceGrantContractsNS['Documents Link'].str.replace('"', '')

In [48]:
dfSalesforceGrantContractsNS.to_excel('test.xlsx')

In [49]:
dfSalesforceGrantContractsSF = dfSalesforceGrantContracts[['Documents Link','Opportunity Name']]

In [50]:
dfSalesforceGrantContractsSF = dfSalesforceGrantContractsSF.drop_duplicates(subset='Opportunity Name', keep="first")

In [51]:
dfSalesforceGrantContractsSF['Documents Link'] = dfSalesforceGrantContractsSF['Documents Link'].str.replace('"', '')

In [52]:
len(dfSalesforce)

9439

In [53]:
dfSalesforce = pd.merge(dfSalesforce, dfSalesforceGrantContractsSF, on = 'Opportunity Name', how='left')

In [54]:
len(dfSalesforce)

9439

In [55]:
#dfSalesforce.to_excel('Output_Salesforce.xlsx')

## Extract Time Allocation From Memo

In [56]:
allocations = ['BONUS ALLOCATION', 'ADMIN ALLOCATION', 'PTO/HOLIDAY ALLOCATION']

In [57]:
all_allocations = '|'.join(allocations)

In [58]:
df1['ALLOCATION'] = df1['Memo'].str.findall(all_allocations).apply(return_first)

## Extract Project Fields for Heather's report

In [59]:
df1['Project Allocations'] = df1['Project'].str.findall(r'\[(.*?)\]').apply(return_first)

## Add Rollup Accounts

In [60]:
lookups = pd.read_excel('../Data/raw/lookups.xlsx')

In [61]:
lookups.head()

,GL Account,Rollup Account
0,5010 Contributions Revenue : Foundation Contri...,5000 Contributions Revenue
1,5015 Contributions Revenue : Campaign Foundati...,5000 Contributions Revenue
2,5020 Contributions Revenue : Individual Contri...,5000 Contributions Revenue
3,5025 Contributions Revenue : Campaign Corporat...,5000 Contributions Revenue
4,5030 Contributions Revenue : Endowment Contrib...,5000 Contributions Revenue


In [62]:
df1 = pd.merge(df1, lookups, on = 'GL Account', how='left')

In [63]:
df1['Account Digit'] = df1['GL Account'].astype('string').str[0]

In [64]:
df1.columns

Index(['Date', 'Period', 'GL Account', 'Document Number', 'Program',
       'Initiative', 'Project', 'Restriction', 'Grant', 'Amount',
       'Vendor Name', 'Employee Name', 'Memo', 'Amount (Credit)',
       'Amount (Debit)', 'Client', 'Transaction Number', 'Type', 'Vendor Rate',
       'Vendor Contract Expiration Date', 'ALLOCATION', 'Project Allocations',
       'Rollup Account', 'Account Digit'],
      dtype='object')

In [65]:
df1['Amount'] = df1['Amount'].astype(float)
df1['Amount (Credit)'].replace('', 0, inplace=True)
df1['Amount (Debit)'].replace('', 0, inplace=True)
df1['Amount (Credit)'] = df1['Amount (Credit)'].astype(float)
df1['Amount (Debit)'] = df1['Amount (Debit)'].astype(float)

In [66]:
df1.dtypes

Date                                object
Period                              object
GL Account                          object
Document Number                     object
Program                             object
Initiative                          object
Project                             object
Restriction                         object
Grant                               object
Amount                             float64
Vendor Name                         object
Employee Name                       object
Memo                                string
Amount (Credit)                    float64
Amount (Debit)                     float64
Client                              object
Transaction Number                  object
Type                                object
Vendor Rate                         object
Vendor Contract Expiration Date     object
ALLOCATION                          object
Project Allocations                 object
Rollup Account                      object
Account Dig

## Add Reclass Field and Closeout field

In [67]:
account_exceptions = ['To reclass', 'Reclass']

In [68]:
Accounting_Moves = '|'.join(account_exceptions)

In [69]:
closeout = 'close out'

In [70]:
df1['Accounting Adjustments'] = df1['Memo'].str.findall(Accounting_Moves).apply(return_first)

In [71]:
df1['Accounting Close Out'] = df1['Memo'].str.findall(closeout).apply(return_first)

In [72]:
df1['Vendor Contract Expiration Date'] = pd.to_datetime(df1['Vendor Contract Expiration Date'])

In [73]:
df1['Vendor Contract Expiration Date'] = df1['Vendor Contract Expiration Date'].dt.date

## ADDING TITLES INSTEAD OF NAMES

In [74]:
#EmployeeNameandID['Job Title'].value_counts()

In [75]:
#uniquetitles = EmployeeNameandID['Job Title'].unique()

In [76]:
#Titleslist = uniquetitles.tolist()

In [77]:
#def createEmployeeTitlesdf(title, df):
#    newdf = df[df['Job Title']==title]
#    newdf.reset_index(drop=True, inplace=True)
#    newdf['PositionID'] = newdf['Job Title'] + newdf.index.astype(str)    
#    return newdf.to_dict()

In [78]:
#DictList = {}

#for i in range(len(Titleslist)):
#    newdict = createEmployeeTitlesdf(Titleslist[i], EmployeeNameandID)
#    DictList[i] = newdict

In [79]:
#dflist = []

#for i in range(len(DictList)):
#    df = pd.DataFrame(DictList[i])
#    dflist.append(df)

In [80]:
#finalemployeedf = pd.concat(dflist).reset_index(drop=True)

In [81]:
#employeelisttomerge = finalemployeedf[['Name', 'PositionID']]

In [82]:
#employeelisttomerge.to_excel('listwithposition.xlsx')

In [83]:
#len(employeelisttomerge)

In [84]:
#df1 = pd.merge(df1, employeelisttomerge, on = 'Name', how='left')

In [85]:
df1['Initiative'].replace('', np.nan, inplace=True)
df1['Project'].replace('', np.nan, inplace=True)
df1['Grant'].replace('', np.nan, inplace=True)
df1['GL Account'].replace('', np.nan, inplace=True)
df1['Memo'].replace('', np.nan, inplace=True)
df1['Transaction Number'].replace('', np.nan, inplace=True)
df1['Vendor Name'].replace('', np.nan, inplace=True)
df1['Vendor Rate'].replace('', np.nan, inplace=True)
df1['Vendor Contract Expiration Date'].replace('', np.nan, inplace=True)
df1['Program'].replace('', np.nan, inplace=True)
df1['Project Allocations'].replace('', np.nan, inplace=True)
df1['Project Allocations'].replace('e', np.nan, inplace=True)
df1['Accounting Adjustments'].replace('To reclass', 'Reclass', inplace=True)
df1['Client'].replace('', np.nan, inplace=True)
df1['Rollup Account'].replace('', np.nan, inplace=True)

c:\users\jeremy wendt\anaconda3\envs\code\detail_trans_project_fy21\notebooks\venv\lib\site-packages\pandas\core\missing.py:49: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask = arr == x


In [86]:
df1['Initiative'].fillna("na", inplace=True)
df1['Project'].fillna("na", inplace=True)
df1['Grant'].fillna("No Grant", inplace=True)
df1['GL Account'].fillna("na", inplace=True)
df1['Memo'].fillna("na", inplace=True)
df1['Transaction Number'].fillna("na", inplace=True)
df1['Vendor Name'].fillna("na", inplace=True)
df1['Vendor Rate'].fillna("na", inplace=True)
df1['Vendor Contract Expiration Date'].fillna("na", inplace=True)
df1['Program'].fillna("na", inplace=True)
df1['Project Allocations'].fillna("Other", inplace=True)
df1['Client'].fillna("na", inplace=True)
df1['Rollup Account'].replace("New Account", np.nan, inplace=True)

## Add Further Heather Calcs

In [87]:
df1['Inst or Reclass'] = df1['Account Digit'] + df1['Accounting Adjustments']

In [88]:
df1['Inst or Reclass'].value_counts()

7           338216
8            55520
5            28076
6            10199
7Reclass      5569
8Reclass      1215
5Reclass       286
6Reclass        23
9                2
Name: Inst or Reclass, dtype: Int64

In [89]:
df1['Inst or Reclass'] = df1['Inst or Reclass'].map({'7': "Expense", '8': "Institutional Overhead", '5': "Revenue", '6': "Revenue", '7Reclass': "Expense Reclass", '8Reclass': "Inst Overhead Reclass", '5Reclass': "Revenue Reclass", '6Reclass': "Revenue Reclass"})

## Unrestricted Columns Added

In [90]:
def return_grant(x):
    if x != "No Grant":
        return "Yes"
    else: 
        return "No"
    
    

In [91]:
df1['grant?'] = df1['Grant'].apply(return_grant)

In [92]:
df1["Restricted Revenue"] = df1['grant?'] + df1['Account Digit']

In [93]:
df1["Restricted Revenue"] = df1["Restricted Revenue"].map({'No7': "No", 'Yes7': "Restricted Revenue", 'No8': "No", 'No5': "No", 'Yes8': "Restricted Revenue", 'Yes6': "No", 'No6': "No", 'Yes5': "No"})

In [94]:
df1['Restricted Revenue'] = np.where(df1['Restricted Revenue'] == "Restricted Revenue", df1['Amount'], 0)

In [95]:
df1["Rev or Exp"] = df1["Account Digit"].map({'6': 'Revenue', '5': 'Revenue', '7': 'Expense', '8': 'Expense'})

In [96]:
df1["Rev or Exp"].value_counts()

Expense    400520
Revenue     38584
Name: Rev or Exp, dtype: int64

In [97]:
df1['Multilateral'] = np.where(df1['Rev or Exp'] == "Revenue", df1['Amount'], 0)

In [98]:
df1['General Expense'] = np.where(df1['Rev or Exp'] == "Expense", df1['Amount'], 0)

In [99]:
df1['Unrestricted'] = - df1['Restricted Revenue'] - df1['Multilateral'] + df1['General Expense']

In [100]:
#df1 = df1.drop('Match ID Names',1)

In [101]:
len(df1)

439108

## Build FFS Tables

In [102]:
df1['Project3YrRev'] = df1.groupby(['Project','Rev or Exp'])['Amount'].transform('sum')

dfrevenuetable = df1[['Project', 'Project3YrRev','Rev or Exp']]

dfrevenuetable = dfrevenuetable[dfrevenuetable['Rev or Exp']=='Revenue']


In [103]:
initiative_program_table = dfMAVELINKACTIVEPROJECTS[['Program', 'Initiative', 'Project', 'Client']]

initiative_program_table['Project-edited'] = initiative_program_table['Client'] + " : " + initiative_program_table['Project']

initiative_program_table = initiative_program_table.drop_duplicates()

initiative_program_table = initiative_program_table[['Program', 'Initiative', 'Project-edited']]

initiative_program_table = initiative_program_table.rename(columns={'Project-edited':'Project'})

<ipython-input-103-5491e83c7e5f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  initiative_program_table['Project-edited'] = initiative_program_table['Client'] + " : " + initiative_program_table['Project']


In [104]:
initiative_program_table2 = df1[['Program', 'Initiative', 'Project']]

In [105]:
initiative_program_table = pd.concat([initiative_program_table, initiative_program_table2], ignore_index=True)

initiative_program_table = initiative_program_table.drop_duplicates()

In [106]:
initiative_program_table

,Program,Initiative,Project
0,Industry,INY - Aviation,WSP : SFO SAF Feasibility
1,India,IN - India,WSP : Microsoft ZE Fleet Initiative
2,China,CHN - Early Peaking Cities,World Wide Fund for Nature Beijing Office - US...
3,Beijing,BEJ - Early Peaking Cities,World Wide Fund for Nature Beijing Office - CN...
4,Operations,OPS - CEO Special Projects,World Resources Institute (WRI) : America's Pl...
...,...,...,...
439670,WattTime,WAT - Admin,"Inspire Energy Holdings, LLC"
439671,WattTime,WAT - Admin,Sapient Industries Inc
439672,WattTime,WAT - Admin,Enerwise Global Technologies LLC WT
439674,WattTime,WAT - Admin,Exelon ComEd Energy Delivery-WT


In [107]:
dfFFSData = dfFFSData.rename(columns={'Name':'Project'})
dfFFSIncludingDeferred = dfFFSIncludingDeferred.rename(columns={'Sum of Total Revenue Recognized':'Total Revenue Recognized'})
dfFFSData2 = dfFFSData2.rename(columns={'Current Month Invoicing Action':'Current Month Action'})
dfFFSData2 = dfFFSData2.rename(columns={'Name':'Project'})

In [108]:
dfFFSData2['DUPLICATE'] = dfFFSData2['Project'].isin(dfFFSData['Project'])

In [109]:
dfFFSData2 = dfFFSData2[dfFFSData2['DUPLICATE']==False]

In [110]:
dfFFSData2 = dfFFSData2[['Project', 'Project Type', 'Contract?', 'Contract Start Date',
       'Contract End Date', 'Contract Amount', 'Invoice Type',
       'Invoicing Notes', 'Current Month Action', 'Project Complete?']]

In [111]:
dfFFSData = pd.concat([dfFFSData, dfFFSData2], ignore_index=True)

In [112]:
dfFFSData = pd.merge(dfFFSData, dfFFSIncludingDeferred, on = 'Project', how='left')


In [113]:
dfFFSData['Total Revenue Recognized'].replace('', np.nan, inplace=True)
dfFFSData['Total Revenue Recognized'].fillna(0, inplace=True)
dfFFSData['Reimbursables NTE'].replace('', np.nan, inplace=True)
dfFFSData['Reimbursables NTE'].fillna(0, inplace=True)
dfFFSData['Invoice Type'].replace('', np.nan, inplace=True)
dfFFSData['Invoice Type'].fillna(0, inplace=True)
dfFFSData['Invoicing Notes'].replace('', np.nan, inplace=True)
dfFFSData['Invoicing Notes'].fillna(0, inplace=True)
dfFFSData['Current Month Action'].replace('', np.nan, inplace=True)
dfFFSData['Current Month Action'].fillna(0, inplace=True)
dfFFSData['Contract Amount'].replace('', np.nan, inplace=True)
dfFFSData['Contract Amount'].fillna(0, inplace=True)
dfFFSData['Remaining Contract Balance'].replace('', np.nan, inplace=True)
dfFFSData['Remaining Contract Balance'].fillna(0, inplace=True)

In [114]:
dfFFSData['Contract Amount'] = dfFFSData['Contract Amount'].astype(float)
dfFFSData['Total Revenue Recognized'] = dfFFSData['Total Revenue Recognized'].astype(float)
dfFFSData['Reimbursables NTE'] = dfFFSData['Reimbursables NTE'].astype(float)
dfFFSData['Reimbursables Actual'] = dfFFSData['Reimbursables Actual'].astype(float)

In [115]:
dfFFSData['Remaining Revenue'] = dfFFSData['Contract Amount'] + dfFFSData['Reimbursables NTE'] - dfFFSData['Total Revenue Recognized']

In [116]:
dfFFSData.loc[dfFFSData['Remaining Revenue'] <= 0, 'Remaining Revenue'] = "na"
dfFFSData.loc[dfFFSData['Contract Amount'] <= 0, 'Contract Amount'] = "na"

In [117]:
dfFFSData['Contract Start Date'] = pd.to_datetime(dfFFSData['Contract Start Date']).dt.strftime("%m/%d/%Y")
dfFFSData['Contract End Date'] = pd.to_datetime(dfFFSData['Contract End Date']).dt.strftime("%m/%d/%Y")

In [118]:
dfFFSDataV2 = dfFFSData

dfFFSDataV2 = pd.merge(dfFFSDataV2, initiative_program_table, on = 'Project', how='left')

dfFFSDataV2 = pd.merge(dfFFSDataV2, dfrevenuetable, on = 'Project', how='left')

dfFFSDataV2 = dfFFSDataV2.drop_duplicates()

In [119]:
dfFFSDataV2 = dfFFSDataV2[dfFFSDataV2['Project'] != "Overall Total"]

In [120]:
dfFFSDataV2['Rev or Exp'].replace('', np.nan, inplace=True)
dfFFSDataV2['Rev or Exp'].fillna(0, inplace=True)
dfFFSDataV2['Reimbursables NTE'].replace('', np.nan, inplace=True)
dfFFSDataV2['Reimbursables NTE'].fillna('na', inplace=True)
dfFFSDataV2['Contract Start Date'].replace('', np.nan, inplace=True)
dfFFSDataV2['Contract Start Date'].fillna('na', inplace=True)
dfFFSDataV2['Contract End Date'].replace('', np.nan, inplace=True)
dfFFSDataV2['Contract End Date'].fillna('na', inplace=True)

In [121]:
PROJECTSWITHOUTINITIATIVES = dfFFSDataV2[pd.isnull(dfFFSDataV2['Initiative'])]

In [122]:
PROJECTSWITHOUTINITIATIVES

,Client,Project,Project Type,Contract?,Contract Start Date,Contract End Date,Invoice Type,Invoicing Notes,Current Month Action,Revenue Recognition Notes,...,Reimbursables NTE,Reimbursables Actual,Remaining Contract Balance,Project Complete?,Total Revenue Recognized,Remaining Revenue,Program,Initiative,Project3YrRev,Rev or Exp
4024,Overall Total,,,,na,na,0,0,0,,...,0.0,1262768.24,0,NaN,0.0,na,NaN,NaN,NaN,0


In [123]:
dfFFSDataV2.to_excel("FFSOpen.xlsx")

In [124]:
dfFFSDataZeros = dfFFSDataV2[dfFFSDataV2['Rev or Exp']==0]

## Add critical datapoints to Netsuite Report from RMI Fee for Service Report 

In [125]:
dfFFSDataedited = dfFFSData[['Project', 'Project Type', 'Contract?',
       'Contract Start Date', 'Contract End Date', 'Invoice Type',
       'Invoicing Notes', 'Current Month Action', 'Revenue Recognition Notes',
       'Contract Amount', 'Total Revenue Recognized', 'Reimbursables NTE',
       'Reimbursables Actual', 'Remaining Revenue']]


dfFFSDataedited.columns

Index(['Project', 'Project Type', 'Contract?', 'Contract Start Date',
       'Contract End Date', 'Invoice Type', 'Invoicing Notes',
       'Current Month Action', 'Revenue Recognition Notes', 'Contract Amount',
       'Total Revenue Recognized', 'Reimbursables NTE', 'Reimbursables Actual',
       'Remaining Revenue'],
      dtype='object')

In [126]:
df1 = pd.merge(df1, dfFFSDataedited, on = 'Project', how='left')

In [127]:
df1.columns

Index(['Date', 'Period', 'GL Account', 'Document Number', 'Program',
       'Initiative', 'Project', 'Restriction', 'Grant', 'Amount',
       'Vendor Name', 'Employee Name', 'Memo', 'Amount (Credit)',
       'Amount (Debit)', 'Client', 'Transaction Number', 'Type', 'Vendor Rate',
       'Vendor Contract Expiration Date', 'ALLOCATION', 'Project Allocations',
       'Rollup Account', 'Account Digit', 'Accounting Adjustments',
       'Accounting Close Out', 'Inst or Reclass', 'grant?',
       'Restricted Revenue', 'Rev or Exp', 'Multilateral', 'General Expense',
       'Unrestricted', 'Project3YrRev', 'Project Type', 'Contract?',
       'Contract Start Date', 'Contract End Date', 'Invoice Type',
       'Invoicing Notes', 'Current Month Action', 'Revenue Recognition Notes',
       'Contract Amount', 'Total Revenue Recognized', 'Reimbursables NTE',
       'Reimbursables Actual', 'Remaining Revenue'],
      dtype='object')

In [128]:
df1['Project Type'].replace('', np.nan, inplace=True)
df1['Contract?'].replace('', np.nan, inplace=True)
df1['Contract Start Date'].replace('', np.nan, inplace=True)
df1['Contract End Date'].replace('', np.nan, inplace=True)
df1['Invoice Type'].replace('', np.nan, inplace=True)
df1['Invoicing Notes'].replace('', np.nan, inplace=True)
df1['Current Month Action'].replace('', np.nan, inplace=True)
df1['Revenue Recognition Notes'].replace('', np.nan, inplace=True)
df1['Contract Amount'].replace('', np.nan, inplace=True)
df1['Total Revenue Recognized'].replace('', np.nan, inplace=True)
df1['Reimbursables NTE'].replace('', np.nan, inplace=True)
df1['Reimbursables Actual'].replace('', inplace=True)
df1['Remaining Revenue'].replace('', np.nan, inplace=True)

In [129]:
df1['Project Type'].fillna("na", inplace=True)
df1['Contract?'].fillna("na", inplace=True)
df1['Contract Start Date'].fillna("na", inplace=True)
df1['Contract End Date'].fillna("na", inplace=True)
df1['Invoice Type'].fillna("na", inplace=True)
df1['Invoicing Notes'].fillna("na", inplace=True)
df1['Current Month Action'].fillna("na", inplace=True)
df1['Revenue Recognition Notes'].fillna("na", inplace=True)
df1['Contract Amount'].fillna("na", inplace=True)
df1['Total Revenue Recognized'].fillna("na", inplace=True)
df1['Reimbursables NTE'].fillna("na", inplace=True)
df1['Reimbursables Actual'].fillna("na", inplace=True)
df1['Remaining Revenue'].fillna("na", inplace=True)

## Grant Balance Data Manipulation

In [130]:
dfGrantBalances = dfGrantBalances[dfGrantBalances['Grant'].notna()]

In [131]:
dfGrantBalancescopy = dfGrantBalances[['Grant', 'Amount Received', 'Total Grant Spend', 'Institutional Overhead Note', 'Accounting Treatment', 'Institutional Overhead', 'NS Balance', 'Grant Close Date (Adaptive Sheet)', 'Grant Start Date', 'Grant Revenue Date', 'Close Date Notes']]

In [132]:
dfGrantBalancescopy['Grant Start Date'] = pd.to_datetime(dfGrantBalancescopy['Grant Start Date']).dt.strftime("%m/%d/%Y")
dfGrantBalancescopy['Grant Revenue Date'] = pd.to_datetime(dfGrantBalancescopy['Grant Revenue Date']).dt.strftime("%m/%d/%Y")
dfGrantBalancescopy['Grant Close Date (Adaptive Sheet)'] = pd.to_datetime(dfGrantBalancescopy['Grant Close Date (Adaptive Sheet)']).dt.strftime("%m/%d/%Y")

<ipython-input-132-13c56aec49e9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfGrantBalancescopy['Grant Start Date'] = pd.to_datetime(dfGrantBalancescopy['Grant Start Date']).dt.strftime("%m/%d/%Y")
<ipython-input-132-13c56aec49e9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfGrantBalancescopy['Grant Revenue Date'] = pd.to_datetime(dfGrantBalancescopy['Grant Revenue Date']).dt.strftime("%m/%d/%Y")
<ipython-input-132-13c56aec49e9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [133]:
dfGrantBalancescopy['Amount Received'] = dfGrantBalancescopy['Amount Received'].astype(float)
dfGrantBalancescopy['Total Grant Spend'] = dfGrantBalancescopy['Total Grant Spend'].astype(float)
dfGrantBalancescopy['Institutional Overhead'] = dfGrantBalancescopy['Institutional Overhead'].astype(float)
dfGrantBalancescopy['NS Balance'] = dfGrantBalancescopy['NS Balance'].astype(float)
#dfGrantBalancescopy['Close Date Notes'] = dfGrantBalancescopy['Close Date Notes'].astype(str)

<ipython-input-133-074b2e46f706>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfGrantBalancescopy['Amount Received'] = dfGrantBalancescopy['Amount Received'].astype(float)
<ipython-input-133-074b2e46f706>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfGrantBalancescopy['Total Grant Spend'] = dfGrantBalancescopy['Total Grant Spend'].astype(float)
<ipython-input-133-074b2e46f706>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [134]:
dfGrantBalancescopy['Grant Close Date (Adaptive Sheet)'].replace('', np.nan, inplace=True)
dfGrantBalancescopy['Grant Start Date'].replace('', np.nan, inplace=True)
dfGrantBalancescopy['Grant Revenue Date'].replace('', np.nan, inplace=True)
dfGrantBalancescopy['Close Date Notes'].replace('', np.nan, inplace=True)
dfGrantBalancescopy['Grant Close Date (Adaptive Sheet)'].fillna("na", inplace=True)
dfGrantBalancescopy['Grant Start Date'].fillna("na", inplace=True)
dfGrantBalancescopy['Grant Revenue Date'].fillna("na", inplace=True)
dfGrantBalancescopy['Close Date Notes'].fillna("na", inplace=True)

c:\users\jeremy wendt\anaconda3\envs\code\detail_trans_project_fy21\notebooks\venv\lib\site-packages\pandas\core\series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
c:\users\jeremy wendt\anaconda3\envs\code\detail_trans_project_fy21\notebooks\venv\lib\site-packages\pandas\core\series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [135]:
df1 = pd.merge(df1, dfGrantBalancescopy, on = 'Grant', how='left')

In [136]:
len(df1)

439108

# Add Hyperlinks

In [137]:
df1 = pd.merge(df1, dfSalesforceGrantContractsNS, on = 'Grant', how='left')

In [138]:
len(df1)

439108

In [139]:
df1['Grant Close Date (Adaptive Sheet)'].replace('', np.nan, inplace=True)
df1['Grant Start Date'].replace('', np.nan, inplace=True)
df1['Close Date Notes'].replace('', np.nan, inplace=True)
df1['Documents Link'].replace('', np.nan, inplace=True)
df1['Grant Revenue Date'].replace('', np.nan, inplace=True)
df1['Amount Received'].replace('', np.nan, inplace=True)
df1['Total Grant Spend'].replace('', np.nan, inplace=True)
df1['Institutional Overhead Note'].replace('', np.nan, inplace=True)
df1['Accounting Treatment'].replace('', np.nan, inplace=True)
df1['Institutional Overhead'].replace('', np.nan, inplace=True)
df1['NS Balance'].replace('', np.nan, inplace=True)
df1['Grant Close Date (Adaptive Sheet)'].fillna("na", inplace=True)
df1['Grant Start Date'].fillna("na", inplace=True)
df1['Grant Revenue Date'].fillna("na", inplace=True)
df1['Amount Received'].fillna("na", inplace=True)
df1['Total Grant Spend'].fillna("na", inplace=True)
df1['Institutional Overhead Note'].fillna("na", inplace=True)
df1['Accounting Treatment'].fillna("na", inplace=True)
df1['Institutional Overhead'].fillna("na", inplace=True)
df1['NS Balance'].fillna("na", inplace=True)
df1['Close Date Notes'].fillna("na", inplace=True)
df1['Documents Link'].fillna("na", inplace=True)

# Create Grant Spenddown of 0 Dataframe

In [140]:
#Missing Project
dfGrantBalancesZEROSPEND = dfGrantBalances[['Level', 'Grant', 'Amount Received', 'Total Grant Spend', 'Institutional Overhead Note', 'Accounting Treatment', 'Institutional Overhead', 'NS Balance', 'Grant Close Date (Adaptive Sheet)', 'Grant Start Date', 'Grant Revenue Date', 'Close Date Notes']]

In [141]:
dfGrantBalancesZEROSPEND = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND['Total Grant Spend']==0]

In [142]:
dfGrantBalancesZEROSPEND["Project"] = ""

In [143]:
dfGrantBalancesZEROSPEND

,Level,Grant,Amount Received,Total Grant Spend,Institutional Overhead Note,Accounting Treatment,Institutional Overhead,NS Balance,Grant Close Date (Adaptive Sheet),Grant Start Date,Grant Revenue Date,Close Date Notes,Project
0,BKT - Third Derivative,FedEx - 3rd Derivative FY21,900000.00,0.0,20% of total cost,NaN,180000.0,720000.00,2024-01-19,NaT,NaT,NaN,
9,GCF - Center for a Climate-Aligned Finance,Hewlett - GCF CCAF FY20,200000.00,0.0,20% of total grant,Revenue Recognized,40000.0,160000.00,2022-06-30,2020-06-01,2020-06-02,NaN,
26,POW - Admin,Heising-Simons - Electricity FY21,250000.00,0.0,0% of Total Cost,NaN,0.0,250000.00,NaT,NaT,NaT,NaN,
27,BLD - Building Electrification,Heising-Simons - Building Electrification FY21,150000.00,0.0,0% of Total Cost,NaN,0.0,150000.00,2021-11-20,2020-12-01,NaT,NaN,
28,GCF - Center for a Climate-Aligned Finance,Climateworks - CAI CCAF FY21,150000.00,0.0,0% of Total Cost,Revenue Recognized,0.0,150000.00,2021-10-31,2020-11-01,2020-12-22,NaN,
30,BLD - Building Electrification,Anonymous - Building Electrification FY21,500000.00,0.0,20% of total grant,Revenue Recognized,66600.0,433400.00,2021-10-14,2020-10-15,2020-11-05,NaN,
33,CIP - Solutions for Oil and Gas,High Tide - Climate Intelligence FY21,250000.00,0.0,15% of total grant,Revenue Recognized,37500.0,212500.00,2021-08-30,2021-03-01,2021-02-03,NaN,
50,UT - City Renewable Accelerator,Schmidt - SolarAPP FY20,50000.00,0.0,20% of total grant,Revenue Recognized,10000.0,40000.00,2021-06-15,2020-06-29,2020-06-29,NaN,
63,GCF - Center for a Climate-Aligned Finance,TD Bank - CCAF FY21,400000.00,0.0,0% of total cost,Revenue Recognized,0.0,400000.00,NaT,NaT,2021-02-22,Anticipated end: FY23 Q2,
64,BKT - Third Derivative,Engie - 3rd Derivative FY21,50000.00,0.0,0% of total cost,0,0.0,50000.00,2024-02-01,NaT,2021-03-12,NaN,


## Output manipulated DF to Excel

In [144]:
%%time

#df1.to_excel('Output_Netsuite.xlsx')

Wall time: 0 ns


# 3.  Data Segmentation by Program

Here I'm taking the manipulated data and ultimately creating a dataframe to filter out each program's restricted revenue.

In [145]:
Programs = list(df1['Program'].unique())

In [146]:
print(Programs)

['Development', 'Global Climate Finance', 'Operations', 'Strategic Engagement and Analysis Group', 'Beijing', 'China', 'Islands', 'Africa', 'Methane', 'Carbon-Free Buildings', 'Sustainable Finance', 'Sunshine for Mines', 'India', 'Carbon-Free Electricity', 'Shine', 'Trucking', 'Carbon-Free Mobility', 'na', 'WattTime', 'Shipping', 'Business Renewable Center', 'Aviation', 'Industry', 'Building Electrification', 'Communications', 'Carbon War Room', 'Emerging Energy Solutions', 'Energy Web Foundation', 'Urban Transformation', 'Breakthrough Technology', 'Climate Aligned Industries', 'Climate Intelligence', 'SE Asia', 'US', 'CFAN', 'Energy Transition Academy', 'Cross-Program Impact Acceleration', 'Canary Media Program']


In [147]:
def create_program(program):
    firstdf = df1[df1['Program']==program]
    seconddf = firstdf[firstdf['Grant'] != 'No Grant'] 
    thirddf = seconddf[(seconddf['Account Digit']=='7')|(seconddf['Account Digit']=='8')]
    if thirddf.empty == True:
        print(str(program)+ " DF is EMPTY")
    else:    
        return thirddf

In [148]:
Development = create_program('Development')
Islands = create_program('Islands')
India = create_program('India')
Communications = create_program('Communications')
Operations = create_program('Operations')
Development = create_program('Development')
China = create_program('China')
Breakthrough_Technology = create_program('Breakthrough Technology')
CarbonFree_Mobility = create_program('Carbon-Free Mobility')
Climate_Aligned_Industries = create_program('Climate Aligned Industries')
Global_Climate_Finance = create_program('Global Climate Finance')
Strategic_Engagement_and_Analysis_Group = create_program('Strategic Engagement and Analysis Group')
CarbonFree_Buildings = create_program('Carbon-Free Buildings')
WattTime = create_program('WattTime')
CarbonFree_Electricty = create_program('Carbon-Free Electricity')
Urban_Transformation = create_program('Urban Transformation')
Climate_Intelligence = create_program('Climate Intelligence')
Industry = create_program('Industry')
SE_Asia = create_program('SE Asia')
Africa = create_program('Africa')
US = create_program('US')
Building_Electrification = create_program('Building Electrification')
Energy_Transition_Academy = create_program('Energy Transition Academy')
Beijing = create_program('Beijing')
CFAN = create_program('CFAN')

Communications DF is EMPTY


In [149]:
Developing_Economies_Rollup = pd.concat([Islands, Africa, SE_Asia, Energy_Transition_Academy, CFAN], ignore_index = True, sort=False)
CarbonFreeBuildings_Rollup = pd.concat([CarbonFree_Buildings, Building_Electrification], ignore_index = True, sort=False)
China_Rollup = pd.concat([China, Beijing], ignore_index = True, sort=False)

## 4. Create Reports Path and Manipulate File Names

I will need to come up with naming conventions that work over time for this project, so in this step I do just that.   Also I am loading the Openpyxl packages that i'm going to need for the rest of this program.

In [150]:
import os

path = r'C:\Users\Jeremy Wendt\anaconda3\envs\Code\Detail_Trans_Project_FY21\reports\Detail_Trans_Reports'

In [151]:
from datetime import datetime
today = datetime.today()
datem = datetime(today.year, today.month-1, 1)
timestamp = "Detail_Transactions_"+datem.strftime("%b_%Y")

In [152]:
print(timestamp+".xlsx")

Detail_Transactions_Apr_2021.xlsx


In [153]:
from openpyxl.styles import Color, PatternFill, Font, Border
greyFill = PatternFill(start_color='00C0C0C0',
                   end_color='00C0C0C0',
                   fill_type='solid')

# Create FFS DFs

In [154]:
def create_df_FeeforService(program):
    firstdf = df1[df1['Program']==program]
    seconddf = firstdf[(firstdf['Account Digit']=='5')|(firstdf['Account Digit']=='6')]
    if seconddf.empty == True:
        print(str(program)+ " DF is EMPTY")
    else:    
        return seconddf

In [155]:
DevelopmentFFS = create_df_FeeforService("Development")
IslandsFFS = create_df_FeeforService("Islands")
IndiaFFS = create_df_FeeforService("India")
OperationsFFS = create_df_FeeforService("Operations")
ChinaFFS = create_df_FeeforService("China")
Breakthrough_Technology_FFS = create_df_FeeforService("Breakthrough Technology")
CarbonFree_MobilityFFS = create_df_FeeforService("Carbon-Free Mobility")
ClimateAligned_IndustriesFFS = create_df_FeeforService("Climate Aligned Industries")
CommunicationsFFS = create_df_FeeforService("Communications")
GlobalClimate_FinanceFFS = create_df_FeeforService("Global Climate Finance")
Strategic_Engagement_and_Analysis_GroupFFS = create_df_FeeforService("Strategic Engagement and Analysis Group")
Carbon_Free_BuildingsFFS = create_df_FeeforService("Carbon-Free Buildings")
Watttime_FFS = create_df_FeeforService("WattTime")
Carbon_Free_ElectricityFFS = create_df_FeeforService("Carbon-Free Electricity")
UrbanTrans_FFS = create_df_FeeforService("Urban Transformation")
ClimateIntelFFS = create_df_FeeforService("Climate Intelligence")
IndustryFFS = create_df_FeeforService("Industry")
SEAsiaFFS = create_df_FeeforService("SE Asia")
AfricaFFS = create_df_FeeforService("Africa")
USFFS = create_df_FeeforService("US")
BuildingElectrificationFFS = create_df_FeeforService("Building Electrification")
Energy_Transition_AcademyFFS = create_df_FeeforService("Energy Transition Academy")
BeijingFFS = create_df_FeeforService("Beijing")
CFANFFS = create_df_FeeforService("CFAN")

In [156]:
Developing_Economies_RollupFFS = pd.concat([IslandsFFS, AfricaFFS, SEAsiaFFS, Energy_Transition_AcademyFFS, CFANFFS], ignore_index = True, sort=False)
CarbonFreeBuildings_RollupFFS = pd.concat([Carbon_Free_BuildingsFFS, BuildingElectrificationFFS], ignore_index = True, sort=False)
China_RollupFFS = pd.concat([ChinaFFS,BeijingFFS], ignore_index = True, sort=False)


# Create All Expense DFs

In [157]:
def create_df_allExpenses(program):
    firstdf = df1[df1['Program']==program]
    seconddf = firstdf[(firstdf['Account Digit']=='7')|(firstdf['Account Digit']=='8')]
    if seconddf.empty == True:
        print(str(program)+ " DF is EMPTY")
    else:    
        return seconddf

In [158]:

DevelopmentExp = create_df_allExpenses("Development")
IslandsExp = create_df_allExpenses("Islands")
IndiaExp = create_df_allExpenses("India")
OperationsExp = create_df_allExpenses("Operations")
ChinaExp = create_df_allExpenses("China")
Breakthrough_Technology_Exp = create_df_allExpenses("Breakthrough Technology")
CarbonFree_MobilityExp = create_df_allExpenses("Carbon-Free Mobility")
ClimateAligned_IndustriesExp = create_df_allExpenses("Climate Aligned Industries")
CommunicationsExp = create_df_allExpenses("Communications")
GlobalClimate_FinanceExp = create_df_allExpenses("Global Climate Finance")
Strategic_Engagement_and_Analysis_GroupExp = create_df_allExpenses("Strategic Engagement and Analysis Group")
Carbon_Free_BuildingsExp = create_df_allExpenses("Carbon-Free Buildings")
Watttime_Exp = create_df_allExpenses("WattTime")
Carbon_Free_ElectricityExp = create_df_allExpenses("Carbon-Free Electricity")
UrbanTrans_Exp = create_df_allExpenses("Urban Transformation")
ClimateIntelExp = create_df_allExpenses("Climate Intelligence")
IndustryExp = create_df_allExpenses("Industry")
SEAsiaExp = create_df_allExpenses("SE Asia")
AfricaExp = create_df_allExpenses("Africa")
USExp = create_df_allExpenses("US")
BuildingElectrificationExp = create_df_allExpenses("Building Electrification")
Energy_Transition_AcademyExp = create_df_allExpenses("Energy Transition Academy")
BeijingExp = create_df_allExpenses("Beijing")
CFANExp = create_df_allExpenses("CFAN")

In [159]:
Developing_Economies_RollupExp = pd.concat([IslandsExp, AfricaExp, SEAsiaExp, Energy_Transition_AcademyExp], ignore_index = True, sort=False)
CarbonFreeBuildings_RollupExp = pd.concat([Carbon_Free_BuildingsExp, BuildingElectrificationExp, CFANExp], ignore_index = True, sort=False)
China_RollupExp = pd.concat([ChinaExp, BeijingExp], ignore_index = True, sort=False)

# Create Reports for Heather and Jon Creyts

In [160]:
todayMD = datetime.today()
datemMD = datetime(today.year, today.month-1, 1)
timestampMD = datem.strftime("%B %Y")

In [161]:
timestampMD

'April 2021'

In [162]:
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment

CFOpath = r'C:\Users\Jeremy Wendt\anaconda3\envs\Code\Detail_Trans_Project_FY21\reports\CFO_Reports'

CFOTemplatepath = 'C:\\Users\\Jeremy Wendt\\anaconda3\\envs\\Code\\Detail_Trans_Project_FY21\\Data\\raw\\CFO_Cover_Page.xlsx'

In [163]:
blueFill = PatternFill(start_color='0033CCCC',
                   end_color='0033CCCC',
                   fill_type='solid')



## Create Cover Sheet CFO

In [164]:
Logopath = r'C:\Users\Jeremy Wendt\anaconda3\envs\Code\Detail_Trans_Project_FY21\notebooks\img\Picture1.png'

# 'C:\\Users\\Jeremy Wendt\\anaconda3\\envs\\Code\\Detail_Trans_Project_FY21\\notebooks\\img\\MD_Cover_Page.xlsx'

In [165]:
wbCFO = load_workbook(CFOTemplatepath)
wsCFO = wbCFO['Cover']    
img = openpyxl.drawing.image.Image(Logopath)
img.anchor = 'A1'
wsCFO.add_image(img)
wsCFO.cell(row = 14, column = 1).value = 'CFO Report ' + timestampMD
wbCFO.save(str(CFOpath+'\\CFO_Report_'+timestampMD+'.xlsx'))

## Restricted Revenue Spend Type by Grant

In [166]:
dfCFOREport = df1[(df1['Period'] == 'Jun 2021')|(df1['Period'] == 'Jul 2020')|(df1['Period'] == 'Aug 2020')|(df1['Period'] == 'Sep 2020')|(df1['Period'] == 'Oct 2020')|(df1['Period'] == 'Nov 2020')|(df1['Period'] == 'Dec 2020')|(df1['Period'] == 'Jan 2021')|(df1['Period'] == 'Feb 2021')|(df1['Period'] == 'Mar 2021')|(df1['Period'] == 'Apr 2021')|(df1['Period'] == 'May 2021')]

In [167]:
def create_CFO_pivot(Group):
    Month_order = ['Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Grant','Inst or Reclass'], values = 'Restricted Revenue', columns = 'Period', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:].sum(axis=1)
    return table2

In [168]:
def add_subtotal_CFO(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k, 'Subtotal'))) for k, d in PivotedGroup.groupby('Grant')]).append((PivotedGroup.sum()).rename(('Grand', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [169]:
def to_excel_CFO(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="RR by Expense Type", startrow = 1)
    writer.save()
    writer.close()   

In [170]:
def edit_workbook_CFO(file):
    book = load_workbook(file)
    sheet = book["RR by Expense Type"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Grant"
    sheet['B2']= "Inst or Reclass" 
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
#    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:O1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['O1']
    top_left_cell.value = "Restricted Revenue by Expense Type"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="003366")
#    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(color="FFFFFF", size = "26")
#    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 25
    sheet.column_dimensions['B'].width = 25
    sheet.column_dimensions['C'].width = 15
    sheet.column_dimensions['D'].width = 15
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 15
    sheet.column_dimensions['K'].width = 15
    sheet.column_dimensions['L'].width = 15
    sheet.column_dimensions['M'].width = 15
    sheet.column_dimensions['N'].width = 15
    sheet.column_dimensions['O'].width = 15
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 2
    for col in range(3, 16):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0;[Red]("$"#,##0)'
    for col in range(1, 3):
        for row in range(3, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=2).value == "Subtotal":
            for j in range(2,16):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))     
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Grand":
            for j in range(1,16):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style="double"))   
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Grant":
            for j in range(1,16):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)  
                sheet.cell(column=j, row=i).fill = greyFill
    for col in range(16, 17):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).border = Border(left=thin)
#    sheet.auto_filter.ref = sheet.dimensions
    sheet.sheet_view.zoomScale = 70
    sheet.freeze_panes = "C3"
    book.save(file)

In [171]:
def format_multi_pivot_CFO(Group, str):
    step1 = create_CFO_pivot(Group)
    step2 = add_subtotal_CFO(step1)
    to_excel_CFO(step2, str)
    edit_workbook_CFO(str)

In [172]:
dfRestrictedONLY = dfCFOREport[dfCFOREport['Rev or Exp']=="Expense"]

In [173]:
format_multi_pivot_CFO(dfRestrictedONLY, CFOpath+'\\CFO_Report_'+timestampMD+'.xlsx')

## Unrestricted Use by Project Allocation and Program

In [174]:
def create_CFO_pivot2(Group):
    Month_order = ['Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Project Allocations','Program'], values = 'Unrestricted', columns = 'Period', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:].sum(axis=1)
    return table2

In [175]:
def add_subtotal_CFO2(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k, 'Subtotal'))) for k, d in PivotedGroup.groupby('Project Allocations')]).append((PivotedGroup.sum()).rename(('Grand', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [176]:
def to_excel_CFO2(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="Unrestricted by Project All", startrow = 1)
    writer.save()
    writer.close()    

In [177]:
def edit_workbook_CFO2(file):
    book = load_workbook(file)
    sheet = book["Unrestricted by Project All"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Project Allocations"
    sheet['B2']= 'Program'
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
#    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:O1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['O1']
    top_left_cell.value = "Unrestricted Used by Project Type"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="003366")
#    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(color="FFFFFF", size = "26")
#    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 25
    sheet.column_dimensions['B'].width = 25
    sheet.column_dimensions['C'].width = 15
    sheet.column_dimensions['D'].width = 15
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 15
    sheet.column_dimensions['K'].width = 15
    sheet.column_dimensions['L'].width = 15
    sheet.column_dimensions['M'].width = 15
    sheet.column_dimensions['N'].width = 15
    sheet.column_dimensions['O'].width = 15
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 2
    for col in range(3, 16):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0;[Red]("$"#,##0)'
    for col in range(1, 3):
        for row in range(3, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=2).value == "Subtotal":
            for j in range(2,16):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))     
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Grand":
            for j in range(1,16):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style="double"))   
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Project Allocations":
            for j in range(1,16):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)  
                sheet.cell(column=j, row=i).fill = greyFill
    for col in range(16, 17):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).border = Border(left=thin)
#    sheet.auto_filter.ref = sheet.dimensions
    sheet.sheet_view.zoomScale = 70
    sheet.freeze_panes = "C3"
    book.save(file)

In [178]:
def format_multi_pivot_CFO2(Group, str):
    step1 = create_CFO_pivot2(Group)
    step2 = add_subtotal_CFO2(step1)
    to_excel_CFO2(step2, str)
    edit_workbook_CFO2(str)

In [179]:
format_multi_pivot_CFO2(dfCFOREport, CFOpath+'\\CFO_Report_'+timestampMD+'.xlsx')

# Managing Director Dashboard

In [180]:
MDpath = r'C:\Users\Jeremy Wendt\anaconda3\envs\Code\Detail_Trans_Project_FY21\reports\MD_Package'


Templatepath = 'C:\\Users\\Jeremy Wendt\\anaconda3\\envs\\Code\\Detail_Trans_Project_FY21\\Data\\raw\\MD_Cover_Page.xlsx'

In [181]:

#wb = openpyxl.load_workbook(filename=Templatepath)
#ws = wb['Cover']

In [182]:
ProgramsMD = Programs

In [183]:
ProgramsMD.remove('na')
ProgramsMD.remove('Building Electrification')
ProgramsMD.remove('Islands')
ProgramsMD.remove('SE Asia')
ProgramsMD.remove('Africa')
ProgramsMD.remove('Energy Transition Academy')
ProgramsMD.remove('Sustainable Finance')
ProgramsMD.remove('Energy Web Foundation')
ProgramsMD.remove('Business Renewable Center')
ProgramsMD.remove('Emerging Energy Solutions')
ProgramsMD.remove('Shine')
ProgramsMD.remove('Carbon War Room')
ProgramsMD.remove('Methane')
ProgramsMD.remove('Sunshine for Mines')
ProgramsMD.remove('Aviation')
ProgramsMD.remove('Trucking')
ProgramsMD.remove('Shipping')
ProgramsMD.append('Developing Economies')
ProgramsMD.remove('Beijing')

In [184]:
ProgramsMD

['Development',
 'Global Climate Finance',
 'Operations',
 'Strategic Engagement and Analysis Group',
 'China',
 'Carbon-Free Buildings',
 'India',
 'Carbon-Free Electricity',
 'Carbon-Free Mobility',
 'WattTime',
 'Industry',
 'Communications',
 'Urban Transformation',
 'Breakthrough Technology',
 'Climate Aligned Industries',
 'Climate Intelligence',
 'US',
 'CFAN',
 'Cross-Program Impact Acceleration',
 'Canary Media Program',
 'Developing Economies']

## Create Cover Sheet

In [185]:
CoverNames = []

for i in ProgramsMD:
    CoverNames.append(i + " - " + timestampMD)

In [186]:
for i in CoverNames:    
    wb = load_workbook(Templatepath)
    ws = wb['Cover']    
    img = openpyxl.drawing.image.Image(Logopath)
    img.anchor = 'A1'
    ws.add_image(img)
    ws.cell(row = 14, column = 1).value = i
    wb.save(str(MDpath+'\\'+i+' - MD Package.xlsx'))

## Create First MD Table

In [187]:
initiative_program_table3 = initiative_program_table[['Program', 'Initiative']].drop_duplicates()

In [188]:
dfGrantBalancesZEROSPEND = dfGrantBalancesZEROSPEND.rename(columns={'Level':'Initiative'})

In [189]:
dfGrantBalancesZEROSPEND = pd.merge(dfGrantBalancesZEROSPEND, initiative_program_table3, on = 'Initiative', how='left')

In [190]:
dfGrantBalancesZEROSPEND = pd.merge(dfGrantBalancesZEROSPEND, dfSalesforceGrantContractsNS, on = 'Grant', how='left')

In [191]:
dfGrantBalancesZEROSPEND.columns

Index(['Initiative', 'Grant', 'Amount Received', 'Total Grant Spend',
       'Institutional Overhead Note', 'Accounting Treatment',
       'Institutional Overhead', 'NS Balance',
       'Grant Close Date (Adaptive Sheet)', 'Grant Start Date',
       'Grant Revenue Date', 'Close Date Notes', 'Project', 'Program',
       'Documents Link'],
      dtype='object')

In [192]:
dfGrantBalancesZEROSPEND['Grant Start Date'] = pd.to_datetime(dfGrantBalancesZEROSPEND['Grant Start Date']).dt.strftime("%m/%d/%Y")
dfGrantBalancesZEROSPEND['Grant Revenue Date'] = pd.to_datetime(dfGrantBalancesZEROSPEND['Grant Revenue Date']).dt.strftime("%m/%d/%Y")
dfGrantBalancesZEROSPEND['Grant Close Date (Adaptive Sheet)'] = pd.to_datetime(dfGrantBalancesZEROSPEND['Grant Close Date (Adaptive Sheet)']).dt.strftime("%m/%d/%Y")
dfGrantBalancesZEROSPEND['Amount Received'] = dfGrantBalancesZEROSPEND['Amount Received'].astype(float)
dfGrantBalancesZEROSPEND['Total Grant Spend'] = dfGrantBalancesZEROSPEND['Total Grant Spend'].astype(float)
dfGrantBalancesZEROSPEND['Institutional Overhead'] = dfGrantBalancesZEROSPEND['Institutional Overhead'].astype(float)
dfGrantBalancesZEROSPEND['NS Balance'] = dfGrantBalancesZEROSPEND['NS Balance'].astype(float)

In [193]:
dfGrantBalancesZEROSPEND['Grant Close Date (Adaptive Sheet)'].replace('', np.nan, inplace=True)
dfGrantBalancesZEROSPEND['Grant Start Date'].replace('', np.nan, inplace=True)
dfGrantBalancesZEROSPEND['Close Date Notes'].replace('', np.nan, inplace=True)
dfGrantBalancesZEROSPEND['Grant Revenue Date'].replace('', np.nan, inplace=True)
dfGrantBalancesZEROSPEND['Amount Received'].replace('', np.nan, inplace=True)
dfGrantBalancesZEROSPEND['Total Grant Spend'].replace('', np.nan, inplace=True)
dfGrantBalancesZEROSPEND['Institutional Overhead Note'].replace('', np.nan, inplace=True)
dfGrantBalancesZEROSPEND['Accounting Treatment'].replace('', np.nan, inplace=True)
dfGrantBalancesZEROSPEND['Institutional Overhead'].replace('', np.nan, inplace=True)
dfGrantBalancesZEROSPEND['NS Balance'].replace('', np.nan, inplace=True)
dfGrantBalancesZEROSPEND['Grant Close Date (Adaptive Sheet)'].fillna("na", inplace=True)
dfGrantBalancesZEROSPEND['Grant Start Date'].fillna("na", inplace=True)
dfGrantBalancesZEROSPEND['Grant Revenue Date'].fillna("na", inplace=True)
dfGrantBalancesZEROSPEND['Amount Received'].fillna("na", inplace=True)
dfGrantBalancesZEROSPEND['Total Grant Spend'].fillna("na", inplace=True)
dfGrantBalancesZEROSPEND['Institutional Overhead Note'].fillna("na", inplace=True)
dfGrantBalancesZEROSPEND['Accounting Treatment'].fillna("na", inplace=True)
dfGrantBalancesZEROSPEND['Institutional Overhead'].fillna("na", inplace=True)
dfGrantBalancesZEROSPEND['NS Balance'].fillna("na", inplace=True)
dfGrantBalancesZEROSPEND['Close Date Notes'].fillna("na", inplace=True)

In [194]:

ChinaGrantNOEXP = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND["Program"]=="China"]
BEGrantNOEXP = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND["Program"]=="Building Electrification"]
CFEGrantNOEXP = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND["Program"]=="Carbon-Free Electricity"]
UTGrantNOEXP = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND["Program"]=="Urban Transformation"]
CAIGrantNOEXP = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND["Program"]=="Climate Aligned Industries"]
CFBGrantNOEXP = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND["Program"]=="Carbon-Free Buildings"]
BTGrantNOEXP = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND["Program"]=="Breakthrough Technology"]
CFMGrantNOEXP = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND["Program"]=="Carbon-Free Mobility"]
ISLANDSGrantNOEXP = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND["Program"]=="Islands"]
IndiaGrantNOEXP = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND["Program"]=="India"]
GCFGrantNOEXP = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND["Program"]=="Global Climate Finance"]
SEAGrantNOEXP = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND["Program"]=="Strategic Engagement and Analysis Group"]
WattTimeGrantNOEXP = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND["Program"]=="WattTime"]
SEASIAGrantNOEXP = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND["Program"]=="SE Asia"]
AfricaGrantNOEXP = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND["Program"]=="Africa"]
USGrantNOEXP = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND["Program"]=="US"]
CIGrantNOEXP = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND["Program"]=="Climate Intelligence"]
ETAGrantNOEXP = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND["Program"]=="Energy Transition Academy"]
CFANGrantNOEXP = dfGrantBalancesZEROSPEND[dfGrantBalancesZEROSPEND["Program"]=="CFAN"]

DEGrantNOEXP = pd.concat([ISLANDSGrantNOEXP, SEASIAGrantNOEXP, AfricaGrantNOEXP, ETAGrantNOEXP, CFANGrantNOEXP], ignore_index = True, sort=False)
CFRRollingGrantNOEXP = pd.concat([BEGrantNOEXP, CFBGrantNOEXP], ignore_index = True, sort=False)

In [195]:
from openpyxl.utils.dataframe import dataframe_to_rows

def create_data_GrantZeroBalance(Group, str):
    table = Group[['Initiative','Project','Grant', 'Documents Link', 'Grant Revenue Date', 'Grant Start Date', 'Grant Close Date (Adaptive Sheet)', 'Close Date Notes', 'Accounting Treatment', 'Amount Received','Total Grant Spend', 'Institutional Overhead Note', 'Institutional Overhead',  'NS Balance']]
    book = load_workbook(MDpath+str)
    sheet = book["Restricted Revenue"]
    data = table
    rows = dataframe_to_rows(table, index=False, header=True)
    for r_idx, row in enumerate(rows, sheet.max_row+2):
        for c_idx, value in enumerate(row, 1):
            sheet.cell(row=r_idx, column=c_idx, value=value)
            sheet.cell(column=c_idx, row=r_idx).font = Font(bold=True)
            sheet.cell(column=c_idx, row=r_idx).number_format = '"$"#,##0;[Red]("$"#,##0)'
            if sheet.cell(column=4, row=r_idx).value is None or sheet.cell(column=4, row=r_idx).value == 'Documents Link':
                continue
            else:
                try:
                    sheet.cell(column=3, row=r_idx).hyperlink = sheet.cell(column=4, row=r_idx).value  
                    sheet.cell(column=3, row=r_idx).value = sheet.cell(column=3, row=r_idx).value    
                    sheet.cell(column=3, row=r_idx).style = "Hyperlink"   
                except:
                    pass    
    book.save(MDpath+str)

In [196]:
def create_MD_pivot1(Group):
    Month_order = ['Jul 2017', 'Aug 2017', 'Sep 2017', 'Oct 2017', 'Nov 2017', 'Dec 2017', 'Jan 2018', 'Feb 2018', 'Mar 2018', 'Apr 2018', 'May 2018', 'Jun 2018', 'Jul 2018', 'Aug 2018', 'Sep 2018', 'Oct 2018', 'Nov 2018', 'Dec 2018', 'Jan 2019', 'Feb 2019', 'Mar 2019', 'Apr 2019', 'May 2019', 'Jun 2019', 'July 2019', 'Aug 2019', 'Sep 2019', 'Oct 2019', 'Nov 2019', 'Dec 2019', 'Jan 2020', 'Feb 2020', 'Mar 2020', 'Apr 2020', 'May 2020', 'Jun 2020', 'Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Initiative','Project','Grant', 'Documents Link', 'Grant Revenue Date', 'Grant Start Date', 'Grant Close Date (Adaptive Sheet)', 'Close Date Notes', 'Accounting Treatment', 'Amount Received','Total Grant Spend', 'Institutional Overhead Note', 'Institutional Overhead',  'NS Balance'], values = 'Amount', columns = 'Period', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['FY21 Total'] = table2.iloc[:,-12:].sum(axis=1)
    table2['FY20-FY21 Total'] = table2.iloc[:,-36:].sum(axis=1)
    table2 = table2[table2['FY20-FY21 Total']!=0]      
    return table2

In [197]:
def add_subtotal_MD1(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k,'','','','','','','','','','','','','Subtotal'))) for k, d in PivotedGroup.groupby('Initiative')]).append((PivotedGroup.sum()).rename(('Grand','','','','','','','','','','','','','Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [198]:
def to_excel_MD1(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="Restricted Revenue", startrow = 1)
    writer.save()
    writer.close()    

In [199]:
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment
from openpyxl.utils import get_column_letter

def edit_workbook_MD1(file):
    book = load_workbook(file)
    sheet = book["Restricted Revenue"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Initiative"
    sheet['B2']= "Project"
    sheet['C2']= "Grant"    
    sheet['D2']= "Grant Agreement Link"       
    sheet['E2']= "Grant Revenue Date"    
    sheet['F2']= "Grant Start Date"    
    sheet['G2']= "Grant Close Date"  
    sheet['H2']= "Close Date Notes" 
    sheet['I2']= "Accounting Treatment "    
    sheet['J2']= "Amount Received"    
    sheet['K2']= "Total Grant Spend"    
    sheet['L2']= "Institutional Overhead Note"    
    sheet['M2']= "Institutional Overhead"    
    sheet['N2']= "Grant Balance"    
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
    sheet['C2'].font = Font(bold=True)
    sheet['D2'].font = Font(bold=True)
    sheet['E2'].font = Font(bold=True)
    sheet['F2'].font = Font(bold=True)
    sheet['G2'].font = Font(bold=True)
    sheet['H2'].font = Font(bold=True)
    sheet['I2'].font = Font(bold=True)
    sheet['J2'].font = Font(bold=True)
    sheet['K2'].font = Font(bold=True)
    sheet['L2'].font = Font(bold=True)
    sheet['M2'].font = Font(bold=True)
#    fontStyle = Font(size = "26")
    sheet.merge_cells('A1:BK1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['BK1']
    top_left_cell.value = "FY21 Restricted Revenue"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="003366")
#    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(color="FFFFFF", size = "26")
#    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 25
    sheet.column_dimensions['B'].width = 25
    sheet.column_dimensions['C'].width = 35
    sheet.column_dimensions['D'].width = 15
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 15
    sheet.column_dimensions['K'].width = 15
    sheet.column_dimensions['L'].width = 15
    sheet.column_dimensions['M'].width = 15
    sheet.column_dimensions['N'].width = 15
    sheet.column_dimensions['O'].width = 15
    sheet.column_dimensions['P'].width = 15
    sheet.column_dimensions['Q'].width = 15
    sheet.column_dimensions['E'].hidden = True
    sheet.column_dimensions['F'].hidden = True  
    sheet.column_dimensions['I'].hidden = True  
    sheet.column_dimensions['BL'].hidden = True
    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 2
    thin_border = Border(left=Side(style='thin'), 
                         right=Side(style='thin'), 
                         top=Side(style='thin'), 
                         bottom=Side(style='thin'))
    for col in range(3, 4):
        for row in range(2, num_rows+1):
            if sheet.cell(column=4, row=row).value == 'na':
                continue
            else:
                try:
                    sheet.cell(column=col, row=row).hyperlink = sheet.cell(column=4, row=row).value  
                    sheet.cell(column=col, row=row).value = sheet.cell(column=col, row=row).value    
                    sheet.cell(column=col, row=row).style = "Hyperlink"   
                except:
                    pass    
    for col in range(3, 4):
        for row in range(2, num_rows+1):
                sheet.cell(column=col, row=row).border = thin_border
    for col in range(9, 64):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0;[Red]("$"#,##0)'
    for col in range(1, 15):
        for row in range(3, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=14).value == "Subtotal":
            for j in range(4,64):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))     
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Grand":
            for j in range(1,64):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style="double"))   
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Initiative":
            for j in range(1,64):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)  
                sheet.cell(column=j, row=i).fill = greyFill
    for col in range(64, 65):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).border = Border(left=thin)
#    sheet.auto_filter.ref = sheet.dimensions
    FullRange = "A2:" + get_column_letter(sheet.max_column) \
    + str(sheet.max_row)
    sheet.auto_filter.ref = FullRange
    sheet.sheet_view.zoomScale = 70
    sheet.freeze_panes = "O3"
    sheet.column_dimensions.group("O","AX",hidden=True)
    book.save(file)

In [200]:
def format_multi_pivot_MD1(Group, str):
    step1 = create_MD_pivot1(Group)
    step2 = add_subtotal_MD1(step1)
    to_excel_MD1(step2, MDpath+str)
    edit_workbook_MD1(MDpath+str)

In [201]:
CoverNames

['Development - April 2021',
 'Global Climate Finance - April 2021',
 'Operations - April 2021',
 'Strategic Engagement and Analysis Group - April 2021',
 'China - April 2021',
 'Carbon-Free Buildings - April 2021',
 'India - April 2021',
 'Carbon-Free Electricity - April 2021',
 'Carbon-Free Mobility - April 2021',
 'WattTime - April 2021',
 'Industry - April 2021',
 'Communications - April 2021',
 'Urban Transformation - April 2021',
 'Breakthrough Technology - April 2021',
 'Climate Aligned Industries - April 2021',
 'Climate Intelligence - April 2021',
 'US - April 2021',
 'CFAN - April 2021',
 'Cross-Program Impact Acceleration - April 2021',
 'Canary Media Program - April 2021',
 'Developing Economies - April 2021']

In [202]:
#!pip install openpyxl==3.0.7

In [203]:
format_multi_pivot_MD1(Developing_Economies_Rollup, '\\Developing Economies - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(Climate_Aligned_Industries, '\\Climate Aligned Industries - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(India, '\\India - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(Operations, '\\Operations - '+timestampMD+' - MD Package.xlsx')
#format_multi_pivot_MD1(Development, '\\Development - '+timestampMD+' - MD Package.xlsx')
#format_multi_pivot_MD1(Communications, '\\Communications - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(China_Rollup, '\\China - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(Breakthrough_Technology, '\\Breakthrough Technology - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(CarbonFree_Mobility, '\\Carbon-Free Mobility - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(CarbonFree_Electricty, '\\Carbon-Free Electricity - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(Strategic_Engagement_and_Analysis_Group, '\\Strategic Engagement and Analysis Group - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(CarbonFreeBuildings_Rollup, '\\Carbon-Free Buildings - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(Urban_Transformation, '\\Urban Transformation - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(Climate_Intelligence, '\\Climate Intelligence - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(Industry, '\\Industry - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(US, '\\US - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(WattTime, '\\WattTime - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD1(Global_Climate_Finance, '\\Global Climate Finance - '+timestampMD+' - MD Package.xlsx')

In [204]:
create_data_GrantZeroBalance(ChinaGrantNOEXP,'\\China - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance(CFEGrantNOEXP,'\\Carbon-Free Electricity - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance(UTGrantNOEXP,'\\Urban Transformation - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance(CAIGrantNOEXP,'\\Climate Aligned Industries - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance(CFRRollingGrantNOEXP,'\\Carbon-Free Buildings - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance(BTGrantNOEXP,'\\Breakthrough Technology - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance(CFMGrantNOEXP,'\\Carbon-Free Mobility - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance(IndiaGrantNOEXP,'\\India - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance(GCFGrantNOEXP,'\\Global Climate Finance - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance(SEAGrantNOEXP,'\\Strategic Engagement and Analysis Group - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance(WattTimeGrantNOEXP,'\\WattTime - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance(USGrantNOEXP,'\\US - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance(CIGrantNOEXP,'\\Climate Intelligence - '+timestampMD+' - MD Package.xlsx')
create_data_GrantZeroBalance(DEGrantNOEXP,'\\Developing Economies - '+timestampMD+' - MD Package.xlsx')


# Create Multi Lat Tab

In [205]:
dfFFSDataZeros = dfFFSDataZeros[['Program','Initiative','Project','Contract?','Project Type', 'Contract Start Date', 'Contract End Date','Invoice Type', 'Invoicing Notes', 'Current Month Action', 'Revenue Recognition Notes', 'Contract Amount', 'Total Revenue Recognized', 'Reimbursables NTE', 'Reimbursables Actual', 'Remaining Revenue']]



ChinaFFSNOREV = dfFFSDataZeros[dfFFSDataZeros["Program"]=="China"]
CFEFFSNOREV = dfFFSDataZeros[dfFFSDataZeros["Program"]=="Carbon-Free Electricity"]
UTFFSNOREV = dfFFSDataZeros[dfFFSDataZeros["Program"]=="Urban Transformation"]
CAIFFSNOREV = dfFFSDataZeros[dfFFSDataZeros["Program"]=="Climate Aligned Industries"]
CFBFFSNOREV = dfFFSDataZeros[dfFFSDataZeros["Program"]=="Carbon-Free Buildings"]
BTFFSNOREV = dfFFSDataZeros[dfFFSDataZeros["Program"]=="Breakthrough Technology"]
CFMFFSNOREV = dfFFSDataZeros[dfFFSDataZeros["Program"]=="Carbon-Free Mobility"]
ISLANDSFFSNOREV = dfFFSDataZeros[dfFFSDataZeros["Program"]=="Islands"]
CIFFSNOREV = dfFFSDataZeros[dfFFSDataZeros["Program"]=="Climate Intelligence"]


In [206]:
from openpyxl.utils.dataframe import dataframe_to_rows

def create_data_FFSZeroBalance(Group, str):
    table = Group[['Initiative','Project','Project Type','Contract?','Contract Start Date', 'Contract End Date','Invoice Type',  'Invoicing Notes', 'Current Month Action', 'Revenue Recognition Notes', 'Contract Amount', 'Total Revenue Recognized', 'Reimbursables NTE', 'Reimbursables Actual', 'Remaining Revenue']]
    book = load_workbook(MDpath+str)
    sheet = book["ER & ML Govt Rev"]
    data = table
    rows = dataframe_to_rows(table, index=False, header=True)
    for r_idx, row in enumerate(rows, sheet.max_row+2):
        for c_idx, value in enumerate(row, 1):
            sheet.cell(row=r_idx, column=c_idx, value=value)
            sheet.cell(column=c_idx, row=r_idx).font = Font(bold=True)
            sheet.cell(column=c_idx, row=r_idx).number_format = '"$"#,##0;[Red]("$"#,##0)'
    book.save(MDpath+str)

In [207]:
def create_MD_pivot2(Group):
    Month_order = ['Jul 2018', 'Aug 2018', 'Sep 2018', 'Oct 2018', 'Nov 2018', 'Dec 2018', 'Jan 2019', 'Feb 2019', 'Mar 2019', 'Apr 2019', 'May 2019', 'Jun 2019', 'July 2019', 'Aug 2019', 'Sep 2019', 'Oct 2019', 'Nov 2019', 'Dec 2019', 'Jan 2020', 'Feb 2020', 'Mar 2020', 'Apr 2020', 'May 2020', 'Jun 2020', 'Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Initiative','Project','Project Type', 'Contract?', 'Contract Start Date', 'Contract End Date','Invoice Type', 'Invoicing Notes', 'Current Month Action', 'Revenue Recognition Notes', 'Contract Amount', 'Total Revenue Recognized', 'Reimbursables NTE', 'Reimbursables Actual', 'Remaining Revenue'], values = 'Amount', columns = 'Period', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['FY21 Total'] = table2.iloc[:,-12:].sum(axis=1)
    table2['FY20-FY21 Total'] = table2.iloc[:,-36:].sum(axis=1)
    table2 = table2[table2['FY20-FY21 Total']!=0]
    return table2

In [208]:
def add_subtotal_MD2(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k, '','','','','','','','','','','','','','Subtotal'))) for k, d in PivotedGroup.groupby('Initiative')]).append((PivotedGroup.sum()).rename(('Grand', '','','','','','','','','','','','','', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [209]:
def to_excel_MD2(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book
   
    df.to_excel(writer, sheet_name="ER & ML Govt Rev", startrow = 1)
    writer.save()
    writer.close()    

In [210]:
def edit_workbook_MD2(file):
    book = load_workbook(file)
    sheet = book["ER & ML Govt Rev"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Initiative"
    sheet['B2']= "Project"
    sheet['C2']= "Project Type"
    sheet['D2']= "Contract?"
    sheet['E2']= "Contract Start Date"
    sheet['F2']= "Contract End Date"
    sheet['G2']= "Invoice Type"
    sheet['H2']= "Invoicing Notes"
    sheet['I2']= "Current Month Action"
    sheet['J2']= "Revenue Recognition Notes"
    sheet['K2']= "Contract Amount"
    sheet['L2']= "Total Revenue Recognized (INCLUDES DEFERRED REVENUE)"
    sheet['M2']= "Reimbursables NTE"
    sheet['N2']= "Reimbursables Actual"
    sheet['O2']= "Remaining Revenue"
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
#    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:AZ1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['AZ1']
    top_left_cell.value = "FY21 Earned, Government, and Multilateral Grant Revenue"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="003366")
#    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(color="FFFFFF", size = "26")
#    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 25
    sheet.column_dimensions['B'].width = 35
    sheet.column_dimensions['C'].width = 10
    sheet.column_dimensions['D'].width = 10
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 15
    sheet.column_dimensions['K'].width = 15
    sheet.column_dimensions['L'].width = 15
    sheet.column_dimensions['M'].width = 15
    sheet.column_dimensions['N'].width = 15
    sheet.column_dimensions['O'].width = 15
    sheet.column_dimensions['E'].hidden = True
    sheet.column_dimensions['I'].hidden = True
    sheet.column_dimensions['M'].hidden = True
    sheet.column_dimensions['N'].hidden = True
    sheet.column_dimensions['J'].hidden = True
    sheet.column_dimensions['BA'].hidden = True
    num_rows = sheet.max_row
    curr_row = 2
    for col in range(1, 16):
        for row in range(3, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for col in range(10, 53):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0;[Red]("$"#,##0)'
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=15).value == "Subtotal":
            for j in range(2,53):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))     
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Grand":
            for j in range(1,53):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style="double"))   
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Initiative":
            for j in range(1,53):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)   
                sheet.cell(column=j, row=i).fill = greyFill
    for col in range(53, 54):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).border = Border(left=thin)
#    sheet.auto_filter.ref = sheet.dimensions
    FullRange = "A2:" + get_column_letter(sheet.max_column) \
    + str(sheet.max_row)
    sheet.auto_filter.ref = FullRange
    sheet.sheet_view.zoomScale = 70
    sheet.freeze_panes = "P3"
    sheet.column_dimensions.group("P","AM",hidden=True)
    book.save(file)

In [211]:
def format_multi_pivot_MD2(Group, str):
    step1 = create_MD_pivot2(Group)
    step2 = add_subtotal_MD2(step1)
    to_excel_MD2(step2, MDpath+str)
    edit_workbook_MD2(MDpath+str)

In [212]:
format_multi_pivot_MD2(Developing_Economies_RollupFFS, '\\Developing Economies - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD2(ClimateAligned_IndustriesFFS, '\\Climate Aligned Industries - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD2(IndiaFFS, '\\India - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD2(OperationsFFS, '\\Operations - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD2(DevelopmentFFS, '\\Development - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD2(CommunicationsFFS, '\\Communications - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD2(ChinaFFS, '\\China - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD2(Breakthrough_Technology_FFS, '\\Breakthrough Technology - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD2(CarbonFree_MobilityFFS, '\\Carbon-Free Mobility - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD2(Carbon_Free_ElectricityFFS, '\\Carbon-Free Electricity - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD2(Strategic_Engagement_and_Analysis_GroupFFS, '\\Strategic Engagement and Analysis Group - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD2(CarbonFreeBuildings_RollupFFS, '\\Carbon-Free Buildings - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD2(UrbanTrans_FFS, '\\Urban Transformation - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD2(ClimateIntelFFS, '\\Climate Intelligence - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD2(IndustryFFS, '\\Industry - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD2(GlobalClimate_FinanceFFS, '\\Global Climate Finance - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD2(Watttime_FFS, '\\WattTime - '+timestampMD+' - MD Package.xlsx')
#format_multi_pivot_MD2(USFFS, '\\US - '+timestampMD+".xlsx")

In [213]:
create_data_FFSZeroBalance(CFEFFSNOREV,'\\Carbon-Free Electricity - '+timestampMD+' - MD Package.xlsx')
create_data_FFSZeroBalance(ChinaFFSNOREV,'\\China - '+timestampMD+' - MD Package.xlsx')
create_data_FFSZeroBalance(UTFFSNOREV,'\\Urban Transformation - '+timestampMD+' - MD Package.xlsx')
create_data_FFSZeroBalance(CAIFFSNOREV,'\\Climate Aligned Industries - '+timestampMD+' - MD Package.xlsx')
create_data_FFSZeroBalance(CFBFFSNOREV,'\\Carbon-Free Buildings - '+timestampMD+' - MD Package.xlsx')
create_data_FFSZeroBalance(BTFFSNOREV,'\\Breakthrough Technology - '+timestampMD+' - MD Package.xlsx')
create_data_FFSZeroBalance(CFMFFSNOREV,'\\Carbon-Free Mobility - '+timestampMD+' - MD Package.xlsx')
create_data_FFSZeroBalance(CIFFSNOREV,'\\Climate Intelligence - '+timestampMD+' - MD Package.xlsx')
create_data_FFSZeroBalance(ISLANDSFFSNOREV,'\\Developing Economies - '+timestampMD+' - MD Package.xlsx')


## Create Total Expenses by Initiative and Project

In [214]:
def create_MD_pivot7(Group):
    Month_order = ['Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Initiative','Project','Rollup Account'], values = 'Amount', columns = 'Period', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:].sum(axis=1)
    return table2

In [215]:
def add_subtotal_MD7(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k,'', 'Subtotal'))) for k, d in PivotedGroup.groupby('Initiative')]).append((PivotedGroup.sum()).rename(('Grand','', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [216]:
def to_excel_MD7(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="Expenses by Project", startrow = 1)
    writer.save()
    writer.close()    

In [217]:
def edit_workbook_MD7(file):
    book = load_workbook(file)
    sheet = book["Expenses by Project"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Initiative"
    sheet['B2']= "Project"
    sheet['C2']= "Account Category"
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
    sheet['C2'].font = Font(bold=True)
#    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:P1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['P1']
    top_left_cell.value = "FY21 Total Program Expenses by Initiative and Project"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="003366")
#    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(color="FFFFFF", size = "26")
#    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 25
    sheet.column_dimensions['B'].width = 25
    sheet.column_dimensions['C'].width = 30
    sheet.column_dimensions['D'].width = 15
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 15
    sheet.column_dimensions['K'].width = 15
    sheet.column_dimensions['L'].width = 15
    sheet.column_dimensions['M'].width = 15
    sheet.column_dimensions['N'].width = 15
    sheet.column_dimensions['O'].width = 15
    sheet.column_dimensions['P'].width = 15
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 2
    for col in range(4, 17):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0;[Red]("$"#,##0)'
    for col in range(1, 4):
        for row in range(3, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=3).value == "Subtotal":
            for j in range(3,17):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))     
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Grand":
            for j in range(1,17):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style="double"))   
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Initiative":
            for j in range(1,17):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)   
                sheet.cell(column=j, row=i).fill = greyFill
    for col in range(17, 18):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).border = Border(left=thin)
#    sheet.auto_filter.ref = sheet.dimensions
    sheet.sheet_view.zoomScale = 70
    sheet.freeze_panes = "D3"
    sheet.sheet_state = 'hidden'
    book.save(file)

In [218]:
def format_multi_pivot_MD7(Group, str):
    step1 = create_MD_pivot7(Group)
    step2 = add_subtotal_MD7(step1)
    to_excel_MD7(step2, MDpath+str)
    edit_workbook_MD7(MDpath+str)

In [219]:
format_multi_pivot_MD7(Developing_Economies_RollupExp, '\\Developing Economies - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD7(ClimateAligned_IndustriesExp, '\\Climate Aligned Industries - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD7(IndiaExp, '\\India - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD7(OperationsExp, '\\Operations - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD7(DevelopmentExp, '\\Development - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD7(CommunicationsExp, '\\Communications - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD7(ChinaExp, '\\China - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD7(Breakthrough_Technology_Exp, '\\Breakthrough Technology - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD7(CarbonFree_MobilityExp, '\\Carbon-Free Mobility - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD7(Carbon_Free_ElectricityExp, '\\Carbon-Free Electricity - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD7(Strategic_Engagement_and_Analysis_GroupExp, '\\Strategic Engagement and Analysis Group - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD7(CarbonFreeBuildings_RollupExp, '\\Carbon-Free Buildings - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD7(UrbanTrans_Exp, '\\Urban Transformation - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD7(ClimateIntelExp, '\\Climate Intelligence - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD7(IndustryExp, '\\Industry - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD7(USExp, '\\US - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD7(GlobalClimate_FinanceExp, '\\Global Climate Finance - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD7(Watttime_Exp, '\\WattTime - '+timestampMD+' - MD Package.xlsx')
#format_multi_pivot_MD3(BuildingElectrificationExp, '\\Building Electrification - '+timestampMD+' - MD Package.xlsx')

# Create All Expenses Report by Initiative and Account

In [220]:
def create_MD_pivot3(Group):
    Month_order = ['Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Initiative','Rollup Account'], values = 'Amount', columns = 'Period', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:].sum(axis=1)
    return table2

In [221]:
def add_subtotal_MD3(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k, 'Subtotal'))) for k, d in PivotedGroup.groupby('Initiative')]).append((PivotedGroup.sum()).rename(('Grand', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [222]:
def to_excel_MD3(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="Total Expenses", startrow = 1)
    writer.save()
    writer.close()    

In [223]:
def edit_workbook_MD3(file):
    book = load_workbook(file)
    sheet = book["Total Expenses"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Initiative"
    sheet['B2']= "Account Category"
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
#    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:O1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['O1']
    top_left_cell.value = "FY21 Total Program Expenses by Initiative and Account"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="003366")
#    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(color="FFFFFF", size = "26")
#    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 25
    sheet.column_dimensions['B'].width = 35
    sheet.column_dimensions['C'].width = 15
    sheet.column_dimensions['D'].width = 15
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 15
    sheet.column_dimensions['K'].width = 15
    sheet.column_dimensions['L'].width = 15
    sheet.column_dimensions['M'].width = 15
    sheet.column_dimensions['N'].width = 15
    sheet.column_dimensions['O'].width = 15
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 2
    for col in range(3, 16):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0;[Red]("$"#,##0)'
    for col in range(1, 3):
        for row in range(3, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=2).value == "Subtotal":
            for j in range(2,16):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))     
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Grand":
            for j in range(1,16):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style="double"))   
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Initiative":
            for j in range(1,16):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)   
                sheet.cell(column=j, row=i).fill = greyFill
    for col in range(16, 17):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).border = Border(left=thin)
#    sheet.auto_filter.ref = sheet.dimensions
    sheet.sheet_view.zoomScale = 70
    sheet.freeze_panes = "C3"
    book.save(file)

In [224]:
def format_multi_pivot_MD3(Group, str):
    step1 = create_MD_pivot3(Group)
    step2 = add_subtotal_MD3(step1)
    to_excel_MD3(step2, MDpath+str)
    edit_workbook_MD3(MDpath+str)

In [225]:
format_multi_pivot_MD3(Developing_Economies_RollupExp, '\\Developing Economies - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(ClimateAligned_IndustriesExp, '\\Climate Aligned Industries - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(IndiaExp, '\\India - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(OperationsExp, '\\Operations - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(DevelopmentExp, '\\Development - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(CommunicationsExp, '\\Communications - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(ChinaExp, '\\China - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(Breakthrough_Technology_Exp, '\\Breakthrough Technology - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(CarbonFree_MobilityExp, '\\Carbon-Free Mobility - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(Carbon_Free_ElectricityExp, '\\Carbon-Free Electricity - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(Strategic_Engagement_and_Analysis_GroupExp, '\\Strategic Engagement and Analysis Group - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(CarbonFreeBuildings_RollupExp, '\\Carbon-Free Buildings - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(UrbanTrans_Exp, '\\Urban Transformation - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(ClimateIntelExp, '\\Climate Intelligence - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(IndustryExp, '\\Industry - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(USExp, '\\US - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(GlobalClimate_FinanceExp, '\\Global Climate Finance - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD3(Watttime_Exp, '\\WattTime - '+timestampMD+' - MD Package.xlsx')
#format_multi_pivot_MD3(BuildingElectrificationExp, '\\Building Electrification - '+timestampMD+' - MD Package.xlsx')

# Unrestricted Report

In [226]:
def create_Totalprogram(program):
    firstdf = df1[df1['Program']==program]
    if firstdf.empty == True:
        print(str(program)+ " DF is EMPTY")
    else:    
        return firstdf

In [227]:
DevelopmentTotal = create_Totalprogram('Development')
CommunicationsTotal = create_Totalprogram('Communications')
IslandsTotal = create_Totalprogram('Islands')
IndiaTotal = create_Totalprogram('India')
OperationsTotal = create_Totalprogram('Operations')
ChinaTotal = create_Totalprogram('China')
Breakthrough_TechnologyTotal = create_Totalprogram('Breakthrough Technology')
CarbonFree_MobilityTotal = create_Totalprogram('Carbon-Free Mobility')
Climate_Aligned_IndustriesTotal = create_Totalprogram('Climate Aligned Industries')
Global_Climate_FinanceTotal = create_Totalprogram('Global Climate Finance')
Strategic_Engagement_and_Analysis_GroupTotal = create_Totalprogram('Strategic Engagement and Analysis Group')
CarbonFree_BuildingsTotal = create_Totalprogram('Carbon-Free Buildings')
WattTimeTotal = create_Totalprogram('WattTime')
CarbonFree_ElectrictyTotal = create_Totalprogram('Carbon-Free Electricity')
Urban_TransformationTotal = create_Totalprogram('Urban Transformation')
Climate_IntelligenceTotal = create_Totalprogram('Climate Intelligence')
IndustryTotal = create_Totalprogram('Industry')
SE_AsiaTotal = create_Totalprogram('SE Asia')
AfricaTotal = create_Totalprogram('Africa')
USTotal = create_Totalprogram('US')
Building_ElectrificationTotal = create_Totalprogram('Building Electrification')
Energy_Transition_AcademyTotal = create_Totalprogram('Energy Transition Academy')
BeijingTotal = create_Totalprogram('Beijing')
CFANTotal = create_Totalprogram('CFAN')

In [228]:
Developing_Economies_RollupTotal = pd.concat([IslandsTotal, AfricaTotal, SE_AsiaTotal, Energy_Transition_AcademyTotal, CFANTotal], ignore_index = True, sort=False)
CarbonFreeBuildings_RollupTotal = pd.concat([CarbonFree_BuildingsTotal, Building_ElectrificationTotal], ignore_index = True, sort=False)
China_RollupTotal = pd.concat([ChinaTotal, BeijingTotal], ignore_index = True, sort=False)

In [229]:
def create_MD_pivot4(Group):
    Month_order = ['Jul 2020', 'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021', 'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021', 'Jun 2021']
    table = pd.pivot_table(Group, index = ['Initiative','Project','Grant'], values = 'Unrestricted', columns = 'Period', aggfunc=np.sum)
    table2 = table.reindex(Month_order, axis=1)
    table2['Total'] = table2.iloc[:,-12:].sum(axis=1)
    return table2

In [230]:
def add_subtotal_MD4(PivotedGroup):
    out = pd.concat([d.append(d.sum().rename((k,'', 'Subtotal'))) for k, d in PivotedGroup.groupby('Initiative')]).append((PivotedGroup.sum()).rename(('Grand','', 'Total')))
    out.index = pd.MultiIndex.from_tuples(out.index)
    return out

In [231]:
def to_excel_MD4(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="Unrestricted Use", startrow = 1)
    writer.save()
    writer.close()    

In [232]:
def edit_workbook_MD4(file):
    book = load_workbook(file)
    sheet = book["Unrestricted Use"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Initiative"
    sheet['B2']= "Project"
    sheet['C2']= "Grant"    
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
    sheet['C2'].font = Font(bold=True)
#    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:P1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['P1']
    top_left_cell.value = "FY21 Unrestricted Use by Initiative, Project, and Grant"
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="003366")
#    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(color="FFFFFF", size = "26")
#    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 25
    sheet.column_dimensions['B'].width = 35
    sheet.column_dimensions['C'].width = 25
    sheet.column_dimensions['D'].width = 15
    sheet.column_dimensions['E'].width = 15
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 15
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 15
    sheet.column_dimensions['K'].width = 15
    sheet.column_dimensions['L'].width = 15
    sheet.column_dimensions['M'].width = 15
    sheet.column_dimensions['N'].width = 15
    sheet.column_dimensions['O'].width = 15
    sheet.column_dimensions['P'].width = 15
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 2
    for col in range(4, 17):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '"$"#,##0;[Red]("$"#,##0)'
    for col in range(1, 4):
        for row in range(3, num_rows):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=3).value == "Subtotal":
            for j in range(3,17):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style='thin'))     
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Grand":
            for j in range(1,17):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(bottom=Side(style="double"))   
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Initiative":
            for j in range(1,17):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin) 
                sheet.cell(column=j, row=i).fill = greyFill
    for col in range(17, 18):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).border = Border(left=thin)
#    sheet.auto_filter.ref = sheet.dimensions
    sheet.sheet_view.zoomScale = 70
    sheet.freeze_panes = "D3"
    book.save(file)

In [233]:
def format_multi_pivot_MD4(Group, str):
    step1 = create_MD_pivot4(Group)
    step2 = add_subtotal_MD4(step1)
    to_excel_MD4(step2, MDpath+str)
    edit_workbook_MD4(MDpath+str)

In [234]:
format_multi_pivot_MD4(Developing_Economies_RollupTotal, '\\Developing Economies - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(Climate_Aligned_IndustriesTotal, '\\Climate Aligned Industries - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(IndiaTotal, '\\India - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(OperationsTotal, '\\Operations - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(DevelopmentTotal, '\\Development - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(CommunicationsTotal, '\\Communications - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(China_RollupTotal, '\\China - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(Breakthrough_TechnologyTotal, '\\Breakthrough Technology - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(CarbonFree_MobilityTotal, '\\Carbon-Free Mobility - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(CarbonFree_ElectrictyTotal, '\\Carbon-Free Electricity - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(Strategic_Engagement_and_Analysis_GroupTotal, '\\Strategic Engagement and Analysis Group - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(CarbonFreeBuildings_RollupTotal, '\\Carbon-Free Buildings - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(Urban_TransformationTotal, '\\Urban Transformation - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(Climate_IntelligenceTotal, '\\Climate Intelligence - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(IndustryTotal, '\\Industry - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(USTotal, '\\US - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(WattTimeTotal, '\\WattTime - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD4(Global_Climate_FinanceTotal, '\\Global Climate Finance - '+timestampMD+' - MD Package.xlsx')


## Create Salesforce Pivots for Philanthropic

In [235]:
def create_Salesforceprogram(program):
    firstdf = dfSalesforce[dfSalesforce['Program: Program Name']==program]
    seconddf = firstdf[firstdf['Grant Status']!='Complete - Satisfactory']
    if seconddf.empty == True:
        print(str(program)+ " DF is EMPTY")
    else:    
        return firstdf

In [236]:
DevelopmentSF = create_Salesforceprogram("General Support")
IslandsSF = create_Salesforceprogram("Islands")
IndiaSF = create_Salesforceprogram("India")
OperationsSF = create_Salesforceprogram("Operations")
ChinaSF = create_Salesforceprogram("China")
Breakthrough_Technology_SF = create_Salesforceprogram("Breakthrough Technology")
CarbonFree_MobilitySF = create_Salesforceprogram("Carbon-Free Mobility")
ClimateAligned_IndustriesSF = create_Salesforceprogram("Climate Aligned Industries")
CommunicationsSF = create_Salesforceprogram("Communications")
GlobalClimate_FinanceSF = create_Salesforceprogram("Global Climate Finance")
Strategic_Engagement_and_Analysis_GroupSF = create_Salesforceprogram("Strategic Engagement and Analysis Group")
Carbon_Free_BuildingsSF = create_Salesforceprogram("Carbon-Free Buildings")
Carbon_Free_ElectricitySF = create_Salesforceprogram("Carbon-Free Electricity")
UrbanTrans_SF = create_Salesforceprogram("Urban Transformation")
ClimateIntelSF = create_Salesforceprogram("Climate Intelligence")
IndustrySF = create_Salesforceprogram("Industry")
SEAsiaSF = create_Salesforceprogram("SE Asia Energy")
AfricaSF = create_Salesforceprogram("Africa")
USSF = create_Salesforceprogram("US")
BuildingElectrificationSF = create_Salesforceprogram("Building Electrification")
Energy_Transition_AcademySF = create_Salesforceprogram("Energy Transition Academy")
BeijingSF = create_Salesforceprogram("Beijing")
CFANSF = create_Salesforceprogram("CFAN")

Communications DF is EMPTY
Building Electrification DF is EMPTY
Beijing DF is EMPTY
CFAN DF is EMPTY


In [237]:
Developing_Economies_RollupSF = pd.concat([IslandsSF, AfricaSF, SEAsiaSF, Energy_Transition_AcademySF, CFANSF], ignore_index = True, sort=False)
CarbonFreeBuildings_RollupSF = pd.concat([Carbon_Free_BuildingsSF, BuildingElectrificationSF], ignore_index = True, sort=False)
China_RollupSF = pd.concat([ChinaSF, BeijingSF], ignore_index = True, sort=False)

In [238]:
def create_MD_pivot5(Group):
    table = pd.pivot_table(Group, index = ['Opportunity Record Type','Initiative','Forecast Category','Opportunity Name', 'Documents Link', 'RMI Grant Name', 'Revenue Name','Stage', 'Revenue Date', 'Grant Term - Start Date', 'Grant Term - End Date', 'Revenue Status', 'Program Fund Type', 'Probability (%)', 'Revenue Amount'], values = 'Expected Revenue', aggfunc=np.sum)
    return table

In [239]:
today = datetime.today()
timestamptoday = today.strftime("%B %d, %Y")

In [240]:
def to_excel_MD5(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="SF Data DEV&GovMLBL", startrow = 1)
    writer.save()
    writer.close()   

In [241]:
def edit_workbook_MD5(file):
    book = load_workbook(file)
    sheet = book["SF Data DEV&GovMLBL"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Opportunity Record Type"    
    sheet['B2']= "Initiative"
    sheet['C2']= "Forecast Category" 
    sheet['D2']= "Opportunity Name"
    sheet['E2']= "Essential Documents Link"      
    sheet['F2']= "RMI Grant Name"    
    sheet['G2']= "Revenue Name" 
    sheet['H2']= "Stage"     
    sheet['I2']= "Revenue Date"     
    sheet['J2']= "Grant Term - Start Date"
    sheet['K2']= "Grant Term - End Date"  
    sheet['L2']= "Revenue Status"   
    sheet['M2']= "Program Fund Type"   
    sheet['N2']= "Probability (%)"     
    sheet['O2']= "Revenue Amount"
    sheet['P2']= "Expected Revenue"   
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
    sheet['C2'].font = Font(bold=True)
    sheet['D2'].font = Font(bold=True) 
#    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:P1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['P1']
    top_left_cell.value = "SalesForce FY20+ Data as of " +timestamptoday
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="003366")
#    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(color="FFFFFF", size = "26")
#    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 25
    sheet.column_dimensions['B'].width = 20
    sheet.column_dimensions['C'].width = 15
    sheet.column_dimensions['D'].width = 30
    sheet.column_dimensions['E'].width = 25
    sheet.column_dimensions['F'].width = 25
    sheet.column_dimensions['G'].width = 30
    sheet.column_dimensions['H'].width = 15
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 15
    sheet.column_dimensions['K'].width = 15
    sheet.column_dimensions['L'].width = 15
    sheet.column_dimensions['M'].width = 15
    sheet.column_dimensions['N'].width = 15
    sheet.column_dimensions['O'].width = 15
    sheet.column_dimensions['F'].hidden = True
    sheet.column_dimensions['E'].hidden = True
    num_rows = sheet.max_row
    curr_row = 2
    thin_border = Border(left=Side(style='thin'), 
                         right=Side(style='thin'), 
                         top=Side(style='thin'), 
                         bottom=Side(style='thin'))
    for col in range(4, 5):
        for row in range(2, num_rows+1):
            if sheet.cell(column=5, row=row).value == 'na':
                continue
            else:
                try:
                    sheet.cell(column=col, row=row).hyperlink = sheet.cell(column=5, row=row).value  
                    sheet.cell(column=col, row=row).value = sheet.cell(column=col, row=row).value    
                    sheet.cell(column=col, row=row).style = "Hyperlink"   
                except:
                    pass    
    for col in range(4, 5):
        for row in range(2, num_rows+1):
                sheet.cell(column=col, row=row).border = thin_border
    for col in range(14, 17):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '#,##0;[Red](#,##0)'
    for col in range(1, 8):
        for row in range(3, num_rows+1):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Opportunity Record Type":
            for j in range(1,17):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)   
                sheet.cell(column=j, row=i).fill = greyFill
    for col in range(16, 17):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).border = Border(top=thin, left=thin, right=thin, bottom=thin)
    for col in range(8, 17):
        for row in range(3, num_rows+1):
            sheet.cell(column=col, row=row).font = Font(bold=False)
    sheet.sheet_view.zoomScale = 85  
    sheet.freeze_panes = sheet['H3']
    book.save(file)

In [242]:
def format_multi_pivot_MD5(Group, str):
    step1 = create_MD_pivot5(Group)
    to_excel_MD5(step1, MDpath+str)
    edit_workbook_MD5(MDpath+str)

In [243]:
format_multi_pivot_MD5(Developing_Economies_RollupSF, '\\Developing Economies - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(ClimateAligned_IndustriesSF, '\\Climate Aligned Industries - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(IndiaSF, '\\India - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(OperationsSF, '\\Operations - '+timestampMD+' - MD Package.xlsx')
#format_multi_pivot_MD5(CommunicationsSF, '\\Communications - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(DevelopmentSF, '\\Development - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(China_RollupSF, '\\China - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(Breakthrough_Technology_SF, '\\Breakthrough Technology - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(CarbonFree_MobilitySF, '\\Carbon-Free Mobility - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(Carbon_Free_ElectricitySF, '\\Carbon-Free Electricity - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(Strategic_Engagement_and_Analysis_GroupSF, '\\Strategic Engagement and Analysis Group - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(CarbonFreeBuildings_RollupSF, '\\Carbon-Free Buildings - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(UrbanTrans_SF, '\\Urban Transformation - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(ClimateIntelSF, '\\Climate Intelligence - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(IndustrySF, '\\Industry - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(USSF, '\\US - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD5(GlobalClimate_FinanceSF, '\\Global Climate Finance - '+timestampMD+' - MD Package.xlsx')
#format_multi_pivot_MD6(BuildingElectrificationSF, '\\Building Electrification - '+timestampMD+' - MD Package.xlsx')

## Create Salesforce Pivots for FFS

In [244]:
def create_Salesforceprogram2(program):
    firstdf = dfSalesforceFFS[dfSalesforceFFS['Program Area: Program Name']==program]
    if firstdf.empty == True:
        print(str(program)+ " DF is EMPTY")
    else:    
        return firstdf

In [245]:
DevelopmentSFffs = create_Salesforceprogram2("General Support")
IslandsSFffs = create_Salesforceprogram2("Islands")
IndiaSFffs = create_Salesforceprogram2("India")
OperationsSFffs = create_Salesforceprogram2("Operations")
ChinaSFffs = create_Salesforceprogram2("China")
Breakthrough_Technology_SFffs = create_Salesforceprogram2("Breakthrough Technology")
CarbonFree_MobilitySFffs = create_Salesforceprogram2("Carbon-Free Mobility")
ClimateAligned_IndustriesSFffs = create_Salesforceprogram2("Climate Aligned Industries")
CommunicationsSF = create_Salesforceprogram("Communications")
GlobalClimate_FinanceSFffs = create_Salesforceprogram2("Global Climate Finance")
Strategic_Engagement_and_Analysis_GroupSFffs = create_Salesforceprogram2("Strategic Engagement and Analysis Group")
Carbon_Free_BuildingsSFffs = create_Salesforceprogram2("Carbon-Free Buildings")
Carbon_Free_ElectricitySFffs = create_Salesforceprogram2("Carbon-Free Electricity")
UrbanTrans_SFffs = create_Salesforceprogram2("Urban Transformation")
ClimateIntelSFffs = create_Salesforceprogram2("Climate Intelligence")
IndustrySFffs = create_Salesforceprogram2("Industry")
SEAsiaSFffs = create_Salesforceprogram2("SE Asia Energy")
AfricaSFffs = create_Salesforceprogram2("Africa")
USSFffs = create_Salesforceprogram2("US")
BuildingElectrificationSFffs = create_Salesforceprogram2("Building Electrification")
Energy_Transition_AcademySFffs = create_Salesforceprogram2("Energy Transition Academy")
BeijingSFffs = create_Salesforceprogram2("Beijing")
CFANSFffs = create_Salesforceprogram2("CFAN")

General Support DF is EMPTY
Operations DF is EMPTY
Communications DF is EMPTY
Building Electrification DF is EMPTY
Energy Transition Academy DF is EMPTY
Beijing DF is EMPTY
CFAN DF is EMPTY


In [246]:
Developing_Economies_RollupSFffs = pd.concat([IslandsSFffs, AfricaSFffs, SEAsiaSFffs, Energy_Transition_AcademySFffs, CFANSFffs], ignore_index = True, sort=False)
CarbonFreeBuildings_RollupSFffs = pd.concat([Carbon_Free_BuildingsSFffs, BuildingElectrificationSFffs], ignore_index = True, sort=False)
China_RollupSFffs = pd.concat([ChinaSFffs, BeijingSFffs], ignore_index = True, sort=False)

In [247]:
def create_MD_pivot6(Group):
    table = pd.pivot_table(Group, index = ['Initiative','Account Name', 'Opportunity Name','Relationship Manager', 'Stage', 'Close Date', 'Probability (%)', 'Amount'], values = 'Weighted Value of Opportunity', aggfunc=np.sum)
    return table

In [248]:
def to_excel_MD6(df, file):
    writer = pd.ExcelWriter(file, engine='openpyxl')
    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book

    df.to_excel(writer, sheet_name="SF Data FFS & Other Opps", startrow = 1)
    writer.save()
    writer.close()   

In [249]:
def edit_workbook_MD6(file):
    book = load_workbook(file)
    sheet = book["SF Data FFS & Other Opps"] #active means last opened sheet
    rd = sheet.row_dimensions[1]
    rd.height = 25
    sheet['A2']= "Initiative"
    sheet['B2']= "Account Name"    
    sheet['C2']= "Opportunity Name"
    sheet['D2']= "Relationship Manager" 
    sheet['E2']= "Stage"
    sheet['F2']= "Close Date"    
    sheet['G2']= "Probability (%)"   
    sheet['H2']= "Amount"  
    sheet['I2']= "Weighted Value of Opportunity"   
    sheet['A2'].font = Font(bold=True)
    sheet['B2'].font = Font(bold=True)
    sheet['C2'].font = Font(bold=True)
    sheet['D2'].font = Font(bold=True) 
#    fontStyle = Font(size = "20")
    sheet.merge_cells('A1:I1')
    top_left_cell = sheet['A1']
    top_right_cell = sheet['I1']
    top_left_cell.value = "SalesForce FY20+ Data as of " +timestamptoday 
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    top_left_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    top_left_cell.fill = PatternFill("solid", fgColor="003366")
#    top_left_cell.fill = fill = GradientFill(stop=("000080", "FFFFFF"))
    top_left_cell.font  = Font(color="FFFFFF", size = "26")
#    top_left_cell.font = fontStyle
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_right_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    sheet.column_dimensions['A'].width = 25
    sheet.column_dimensions['B'].width = 25
    sheet.column_dimensions['C'].width = 35
    sheet.column_dimensions['D'].width = 25
    sheet.column_dimensions['E'].width = 25
    sheet.column_dimensions['F'].width = 15
    sheet.column_dimensions['G'].width = 25
    sheet.column_dimensions['H'].width = 25
    sheet.column_dimensions['I'].width = 15
    sheet.column_dimensions['J'].width = 15
#    sheet.column_dimensions['D'].hidden = True
    num_rows = sheet.max_row
    curr_row = 2
    for col in range(7, 10):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).number_format = '#,##0;[Red](#,##0)'
    for col in range(1, 6):
        for row in range(3, num_rows+1):
            sheet.cell(column=col, row=row).alignment = Alignment(horizontal = 'left', vertical='top')
            sheet.cell(column=col, row=row).fill = blueFill
    for i in range(1, sheet.max_row + 1):
        if sheet.cell(row=i, column=1).value == "Initiative":
            for j in range(1,10):
                sheet.cell(column=j, row=i).font = Font(bold=True)
                sheet.cell(column=j, row=i).border = Border(top = thin, bottom = thin)   
                sheet.cell(column=j, row=i).fill = greyFill
    for col in range(9, 10):
        for row in range(2, num_rows+1):
            sheet.cell(column=col, row=row).border = Border(top=thin, left=thin, right=thin, bottom=thin)
    for col in range(6, 10):
        for row in range(3, num_rows+1):
            sheet.cell(column=col, row=row).font = Font(bold=False)
    sheet.sheet_view.zoomScale = 70  
    sheet.freeze_panes = sheet['F3']
    book.save(file)

In [250]:
def format_multi_pivot_MD6(Group, str):
    step1 = create_MD_pivot6(Group)
    to_excel_MD6(step1, MDpath+str)
    edit_workbook_MD6(MDpath+str)

In [251]:
format_multi_pivot_MD6(Developing_Economies_RollupSFffs, '\\Developing Economies - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD6(ClimateAligned_IndustriesSFffs, '\\Climate Aligned Industries - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD6(IndiaSFffs, '\\India - '+timestampMD+' - MD Package.xlsx')
#format_multi_pivot_MD6(OperationsSFffs, '\\Operations - '+timestampMD+' - MD Package.xlsx')
#format_multi_pivot_MD6(DevelopmentSFffs, '\\Development - '+timestampMD+' - MD Package.xlsx')
#format_multi_pivot_MD6(CommunicationsSFffs, '\\Communications - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD6(China_RollupSFffs, '\\China - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD6(Breakthrough_Technology_SFffs, '\\Breakthrough Technology - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD6(CarbonFree_MobilitySFffs, '\\Carbon-Free Mobility - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD6(Carbon_Free_ElectricitySFffs, '\\Carbon-Free Electricity - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD6(Strategic_Engagement_and_Analysis_GroupSFffs, '\\Strategic Engagement and Analysis Group - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD6(CarbonFreeBuildings_RollupSFffs, '\\Carbon-Free Buildings - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD6(UrbanTrans_SFffs, '\\Urban Transformation - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD6(ClimateIntelSFffs, '\\Climate Intelligence - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD6(IndustrySFffs, '\\Industry - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD6(USSFffs, '\\US - '+timestampMD+' - MD Package.xlsx')
format_multi_pivot_MD6(GlobalClimate_FinanceSFffs, '\\Global Climate Finance - '+timestampMD+' - MD Package.xlsx')